# General instructions

The goal of the project is to materialize a set of **exploratory workloads** over a real-world, large-scale,  open-domain KG: [WikiData](https://www.wikidata.org/wiki/Wikidata:Main_Page)

An exploratory workload  is composed by a set of queries, where each query is related to the information obtained previously.

An exploratory workload starts with a usually vague, open ended question, and does not assume the person issuing the workload has a clear understanding of the data contained in the target database or its structure.

Remeber that:

1. All the queries must run in the python notebook
2. You can use classes and properties only if you find them via a SPARQL query that must be present in the notebook
3. You do not delete useless queries. Keep everything that is synthatically valid 

```
?p <http://schema.org/name> ?name .
```
    
is the BGP returning a human-readable name of a property or a class in Wikidata.
    
    

In [1]:
## SETUP used later

from SPARQLWrapper import SPARQLWrapper, JSON


prefixString = """
##-567b0d5169-##
PREFIX wd: <http://www.wikidata.org/entity/> 
PREFIX wdt: <http://www.wikidata.org/prop/direct/> 
PREFIX sc: <http://schema.org/>
"""

# select and construct queries
def run_query(queryString):
    to_run = prefixString + "\n" + queryString

    sparql = SPARQLWrapper("http://a256-gc1-02.srv.aau.dk:5820/sparql")
    sparql.setTimeout(300)
    sparql.setReturnFormat(JSON)
    sparql.setQuery(to_run)

    try :
       results = sparql.query()
       json_results = results.convert()
       if len(json_results['results']['bindings'])==0:
          print("Empty")
          return 0
    
       for bindings in json_results['results']['bindings']:
          print( [ (var, value['value'])  for var, value in bindings.items() ] )

       return len(json_results['results']['bindings'])

    except Exception as e :
        print("The operation failed", e)
    
# ASk queries
def run_ask_query(queryString):
    to_run = prefixString + "\n" + queryString

    sparql = SPARQLWrapper("http://a256-gc1-02.srv.aau.dk:5820/sparql")
    sparql.setTimeout(300)
    sparql.setReturnFormat(JSON)
    sparql.setQuery(to_run)

    try :
        return sparql.query().convert()

    except Exception as e :
        print("The operation failed", e)


# Sport Workflow Series ("Olympic explorative search") 

Consider the following exploratory information need:

> investigate the results, in terms of participations and competitions won, in the past editions of the olympics between France and Germany

## Useful URIs for the current workflow


The following are given:

| IRI           | Description   | Role      |
| -----------   | -----------   |-----------|
| `wdt:P1647`   | subproperty   | predicate |
| `wdt:P31`     | instance of   | predicate |
| `wdt:P106`    | profession    | predicate | 
| `wdt:P279`    | subclass      | predicate |
| `wdt:P27`     | nationality   | predicate |
| `wd:Q159821`  | Olympic games | node      |
| `wd:Q10843402`| Swimmer       | node |
| `wd:Q2066131` | Athlete       | node |
| `wd:Q142`     | France        | node |
| `wd:Q183`     | Germany       | node |


Also consider


```
?a wdt:P106/wdt:P279 wd:Q2066131
```

is the BGP to retrieve all instances of **athlete**


```
?p wdt:P27 wd:Q142
```

is the BGP to retrieve all **citizens of France**

## Workload Goals

1. Identify the BGP for participating or winning some competition

2. Identify the BGP for different sports or types of competitions

3. Is there some French or German Athlete participating to any olympic event with double citizenship?

4. Compare number of participations and winning events across different dimensions
 
   4.1 In which specific sport France has more winners than Germany
   
   4.2 Who has the highest number of female winners
   
   4.3 For each country , which are the atheletes with the highest number of participations


In [1]:
# start your workflow here

In [2]:
queryString = """
SELECT COUNT(*)
WHERE { 

?a wdt:P106/wdt:P279 wd:Q2066131

} 
"""

print("Results")
run_query(queryString)

Results
[('callret-0', '929008')]


1

<h3> TASK 1 STARTS HERE </h3>
Identify the BGP for participating or winning some competition.

Retrieve some properties of a french athlete.

In [3]:
#Q1
#get the properties of a french athlete
queryString = """
SELECT DISTINCT ?p ?pname
WHERE {
    ?a wdt:P106/wdt:P279 wd:Q2066131;
       wdt:P27 wd:Q142;
    ?p ?o .

    #labels
    ?p <http://schema.org/name> ?pname .
}
LIMIT 50
"""
print("Results")
run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P4588'), ('pname', 'International World Games Association athlete ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1006'), ('pname', 'Nationale Thesaurus voor Auteurs ID')]
[('p', 'http://www.wikidata.org/prop/direct/P103'), ('pname', 'native language')]
[('p', 'http://www.wikidata.org/prop/direct/P106'), ('pname', 'occupation')]
[('p', 'http://www.wikidata.org/prop/direct/P1207'), ('pname', 'NUKAT ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1344'), ('pname', 'participant in')]
[('p', 'http://www.wikidata.org/prop/direct/P1412'), ('pname', 'languages spoken, written or signed')]
[('p', 'http://www.wikidata.org/prop/direct/P1532'), ('pname', 'country for sport')]
[('p', 'http://www.wikidata.org/prop/direct/P18'), ('pname', 'image')]
[('p', 'http://www.wikidata.org/prop/direct/P1889'), ('pname', 'different from')]
[('p', 'http://www.wikidata.org/prop/direct/P19'), ('pname', 'place of birth')]
[('p', 'http://www.wikidata.org/pro

50

I am trying to find the properties that connect an athlete to an instance of Olympics.

In [4]:
#Q2
#get the properties of a french athlete that connect to Olympics
queryString = """
SELECT DISTINCT ?p ?pname
WHERE {
    ?a wdt:P106/wdt:P279 wd:Q2066131;
       wdt:P27 wd:Q142;
    ?p ?o .
    
    ?o wdt:P31 wd:Q159821.

    #labels
    ?p <http://schema.org/name> ?pname .
}
LIMIT 50
"""
print("Results")
run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1344'), ('pname', 'participant in')]
[('p', 'http://www.wikidata.org/prop/direct/P2522'), ('pname', 'victory')]
[('p', 'http://www.wikidata.org/prop/direct/P710'), ('pname', 'participant')]
[('p', 'http://www.wikidata.org/prop/direct/P793'), ('pname', 'significant event')]


4

Retrieve some competitions where an athlete took part in.

In [5]:
#Q3
#get the objects with french athlete as subject and partecipant in (P1344) as property
queryString = """
SELECT DISTINCT ?o ?oname
WHERE {
    ?a wdt:P106/wdt:P279 wd:Q2066131;
       wdt:P27 wd:Q142;
       wdt:P1344 ?o.
       
    #labels
    ?o <http://schema.org/name> ?oname .
}
LIMIT 30
"""
print("Results")
run_query(queryString)

Results
[('o', 'http://www.wikidata.org/entity/Q11831154'), ('oname', '2019 Africa Cup of Nations')]
[('o', 'http://www.wikidata.org/entity/Q1239608'), ('oname', '2011–12 UCI America Tour')]
[('o', 'http://www.wikidata.org/entity/Q1280387'), ('oname', '2013 UCI Europe Tour')]
[('o', 'http://www.wikidata.org/entity/Q15041668'), ('oname', '2014 UCI Europe Tour')]
[('o', 'http://www.wikidata.org/entity/Q18342122'), ('oname', '2015 UCI Europe Tour')]
[('o', 'http://www.wikidata.org/entity/Q21029681'), ('oname', '2016 UCI Europe Tour')]
[('o', 'http://www.wikidata.org/entity/Q30750603'), ('oname', '2017 UCI World Ranking')]
[('o', 'http://www.wikidata.org/entity/Q751208'), ('oname', '2010–11 UCI Europe Tour')]
[('o', 'http://www.wikidata.org/entity/Q8150'), ('oname', '1936 Summer Olympics')]
[('o', 'http://www.wikidata.org/entity/Q21857932'), ('oname', '2017 UCI World Tour')]
[('o', 'http://www.wikidata.org/entity/Q27915850'), ('oname', '2017 UCI Europe Tour')]
[('o', 'http://www.wikidata.o

30

Same query as above but for an athlete in general (no nationality constraint).

In [8]:
#Q4
#try (P1344=participant in) without 'french athlete' and see what competitions are retrived
queryString = """
SELECT DISTINCT ?o ?oname
WHERE {
    ?a wdt:P1344 ?o.
       
    #labels
    ?o <http://schema.org/name> ?oname .
}
LIMIT 50
"""
print("Results")
run_query(queryString)

Results
[('o', 'http://www.wikidata.org/entity/Q1053559'), ('oname', '2020 Winter Youth Olympics')]
[('o', 'http://www.wikidata.org/entity/Q1082132'), ('oname', '2004 UEFA European Under-17 Football Championship')]
[('o', 'http://www.wikidata.org/entity/Q1209347'), ('oname', '1987 Southeast Asian Games')]
[('o', 'http://www.wikidata.org/entity/Q1279521'), ('oname', '2013 Copa Libertadores')]
[('o', 'http://www.wikidata.org/entity/Q129304'), ('oname', '2013 CONCACAF U-20 Championship')]
[('o', 'http://www.wikidata.org/entity/Q162977'), ('oname', 'Egypt')]
[('o', 'http://www.wikidata.org/entity/Q1649567'), ('oname', '1965 Major League Baseball season')]
[('o', 'http://www.wikidata.org/entity/Q166551'), ('oname', '1997 FIFA Confederations Cup')]
[('o', 'http://www.wikidata.org/entity/Q216330'), ('oname', '24 Hours of Daytona')]
[('o', 'http://www.wikidata.org/entity/Q2365546'), ('oname', '2011 CONCACAF U-17 Championship')]
[('o', 'http://www.wikidata.org/entity/Q244367'), ('oname', '2007–

50

Run the same query as above but using P710 (participant) to see if there are some differences.

In [5]:
#Q5
#run other property to figure if there is any difference(P710=participant) 
queryString = """
SELECT DISTINCT ?o ?oname
WHERE {
    ?a wdt:P106/wdt:P279 wd:Q2066131;
       wdt:P27 wd:Q142;
       wdt:P710 ?o.
       
    #labels
    ?o <http://schema.org/name> ?oname .
}
LIMIT 30
"""
print("Results")
run_query(queryString)

Results
[('o', 'http://www.wikidata.org/entity/Q28739880'), ('oname', 'Eastern Mediterranean Championship Tennis')]
[('o', 'http://www.wikidata.org/entity/Q27779355'), ('oname', "Panhellenic men's and women's tennis championship")]
[('o', 'http://www.wikidata.org/entity/Q775871'), ('oname', 'football at the 1906 Intercalated Games')]
[('o', 'http://www.wikidata.org/entity/Q1069341'), ('oname', 'tennis at the 1906 Intercalated Games')]
[('o', 'http://www.wikidata.org/entity/Q8567'), ('oname', '2008 Summer Olympics')]
[('o', 'http://www.wikidata.org/entity/Q4630361'), ('oname', "2019 FIFA Women's World Cup")]
[('o', 'http://www.wikidata.org/entity/Q1344963'), ('oname', 'world championship')]
[('o', 'http://www.wikidata.org/entity/Q249097'), ('oname', '1906 Tour de France')]
[('o', 'http://www.wikidata.org/entity/Q634350'), ('oname', '1910 Tour de France')]
[('o', 'http://www.wikidata.org/entity/Q734077'), ('oname', '1905 Tour de France')]
[('o', 'http://www.wikidata.org/entity/Q2202752')

20

Retrieve some properties of competitions.

In [8]:
#Q6
#get labels of some competitions (P710=participant) 
queryString = """
SELECT DISTINCT ?p ?pname
WHERE {
    ?a wdt:P106/wdt:P279 wd:Q2066131;
       wdt:P27 wd:Q142;
       wdt:P710 ?c.
    
    ?c ?p ?o .
       
    #labels
    ?p <http://schema.org/name> ?pname .
}
LIMIT 50
"""
print("Results")
run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1132'), ('pname', 'number of participants')]
[('p', 'http://www.wikidata.org/prop/direct/P155'), ('pname', 'follows')]
[('p', 'http://www.wikidata.org/prop/direct/P156'), ('pname', 'followed by')]
[('p', 'http://www.wikidata.org/prop/direct/P17'), ('pname', 'country')]
[('p', 'http://www.wikidata.org/prop/direct/P2031'), ('pname', 'work period (start)')]
[('p', 'http://www.wikidata.org/prop/direct/P2032'), ('pname', 'work period (end)')]
[('p', 'http://www.wikidata.org/prop/direct/P2671'), ('pname', 'Google Knowledge Graph ID')]
[('p', 'http://www.wikidata.org/prop/direct/P276'), ('pname', 'location')]
[('p', 'http://www.wikidata.org/prop/direct/P279'), ('pname', 'subclass of')]
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('pname', 'instance of')]
[('p', 'http://www.wikidata.org/prop/direct/P3450'), ('pname', 'sports season of league or competition')]
[('p', 'http://www.wikidata.org/prop/direct/P361'), ('pname', 'part of')]
[(

50

As above but excluding literals.

In [9]:
#Q7
#get labels (exclude literals) of some competitions (P710=participant) 
queryString = """
SELECT DISTINCT ?p ?pname
WHERE {
    ?a wdt:P106/wdt:P279 wd:Q2066131;
       wdt:P27 wd:Q142;
       wdt:P710 ?c.
    
    ?c ?p ?o . FILTER(!isLiteral(?o))
       
    #labels
    ?p <http://schema.org/name> ?pname .
}
LIMIT 50
"""
print("Results")
run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P155'), ('pname', 'follows')]
[('p', 'http://www.wikidata.org/prop/direct/P156'), ('pname', 'followed by')]
[('p', 'http://www.wikidata.org/prop/direct/P17'), ('pname', 'country')]
[('p', 'http://www.wikidata.org/prop/direct/P276'), ('pname', 'location')]
[('p', 'http://www.wikidata.org/prop/direct/P279'), ('pname', 'subclass of')]
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('pname', 'instance of')]
[('p', 'http://www.wikidata.org/prop/direct/P3450'), ('pname', 'sports season of league or competition')]
[('p', 'http://www.wikidata.org/prop/direct/P361'), ('pname', 'part of')]
[('p', 'http://www.wikidata.org/prop/direct/P527'), ('pname', 'has part')]
[('p', 'http://www.wikidata.org/prop/direct/P641'), ('pname', 'sport')]
[('p', 'http://www.wikidata.org/prop/direct/P664'), ('pname', 'organizer')]
[('p', 'http://www.wikidata.org/prop/direct/P710'), ('pname', 'participant')]
[('p', 'http://www.wikidata.org/prop/direct/P840'), ('pn

47

I am trying to use P1346 (winner). Retrieve the competitions won by a french athlete.

In [10]:
#Q8
#add the property winner(P1346) found in the cell above
queryString = """
SELECT DISTINCT ?c ?cname
WHERE {
    ?a wdt:P106/wdt:P279 wd:Q2066131;
       wdt:P27 wd:Q142;
       wdt:P1346 ?c.
       
    #labels
    ?c <http://schema.org/name> ?cname .
}
LIMIT 50
"""
print("Results")
run_query(queryString)

Results
Empty


0

Remove the nationality constraint to see if we get some results.

In [11]:
#Q9
#same as before but without 'french' to enlarge the view
queryString = """
SELECT DISTINCT ?c ?cname
WHERE {
    ?a wdt:P106/wdt:P279 wd:Q2066131;
       wdt:P1346 ?c.
       
    #labels
    ?c <http://schema.org/name> ?cname .
}
LIMIT 50
"""
print("Results")
run_query(queryString)

Results
[('c', 'http://www.wikidata.org/entity/Q57609226'), ('cname', "2018 World Wrestling Championships – Women's freestyle 55 kg")]
[('c', 'http://www.wikidata.org/entity/Q28406503'), ('cname', "2016 World Wrestling Championships – women's freestyle 55 kg")]
[('c', 'http://www.wikidata.org/entity/Q478328'), ('cname', 'Amsterdam Marathon')]
[('c', 'http://www.wikidata.org/entity/Q752138'), ('cname', 'New York City Marathon')]
[('c', 'http://www.wikidata.org/entity/Q1628588'), ('cname', 'North American Boxing Federation')]
[('c', 'http://www.wikidata.org/entity/Q1209340'), ('cname', '2007 Southeast Asian Games')]
[('c', 'http://www.wikidata.org/entity/Q24908675'), ('cname', '2016 Supercopa de España de Baloncesto')]
[('c', 'http://www.wikidata.org/entity/Q19323'), ('cname', "FIFA Women's World Cup")]
[('c', 'http://www.wikidata.org/entity/Q11680472'), ('cname', 'Galician Rally Championship')]
[('c', 'http://www.wikidata.org/entity/Q1059464'), ('cname', 'WWE Raw Tag Team Championship')

15

Try to use P2522 (victory) instead of P1346 to check if we can get more results.

In [12]:
#Q10
#check the difference with P2522 'victory' (found in q4)
queryString = """
SELECT DISTINCT ?c ?cname
WHERE {
    ?a wdt:P106/wdt:P279 wd:Q2066131;
       wdt:P2522 ?c.
       
    #labels
    ?c <http://schema.org/name> ?cname .
}
LIMIT 100
"""
print("Results")
run_query(queryString)

Results
[('c', 'http://www.wikidata.org/entity/Q1848041'), ('cname', 'European Squash Individual Championships')]
[('c', 'http://www.wikidata.org/entity/Q28132075'), ('cname', 'Monte-Carlo Squash Classic 2013')]
[('c', 'http://www.wikidata.org/entity/Q17009124'), ('cname', "2014 Women's World Team Squash Championships")]
[('c', 'http://www.wikidata.org/entity/Q20711297'), ('cname', "Women's China Squash Open 2015")]
[('c', 'http://www.wikidata.org/entity/Q1995390'), ('cname', '1993 BDO World Darts Championship')]
[('c', 'http://www.wikidata.org/entity/Q2195114'), ('cname', '1987 BDO World Darts Championship')]
[('c', 'http://www.wikidata.org/entity/Q2742604'), ('cname', '1982 BDO World Darts Championship')]
[('c', 'http://www.wikidata.org/entity/Q2186851'), ('cname', '1981 BDO World Darts Championship')]
[('c', 'http://www.wikidata.org/entity/Q2189461'), ('cname', '1980 BDO World Darts Championship')]
[('c', 'http://www.wikidata.org/entity/Q2420623'), ('cname', '1984 BDO World Darts Ch

100

In [10]:
#Q11
#check winner (2522) without the athlete constraint
queryString = """
SELECT DISTINCT ?c ?cname
WHERE {
    ?a wdt:P2522 ?c.
       
    #labels
    ?c <http://schema.org/name> ?cname .
}
LIMIT 50
"""
print("Results")
run_query(queryString)

Results
[('c', 'http://www.wikidata.org/entity/Q1082132'), ('cname', '2004 UEFA European Under-17 Football Championship')]
[('c', 'http://www.wikidata.org/entity/Q1785470'), ('cname', '1999 Major League Soccer season')]
[('c', 'http://www.wikidata.org/entity/Q1886152'), ('cname', '2006 Major League Soccer season')]
[('c', 'http://www.wikidata.org/entity/Q216330'), ('cname', '24 Hours of Daytona')]
[('c', 'http://www.wikidata.org/entity/Q244367'), ('cname', '2007–08 La Liga')]
[('c', 'http://www.wikidata.org/entity/Q2686578'), ('cname', '1976 Davis Cup')]
[('c', 'http://www.wikidata.org/entity/Q3066961'), ('cname', 'Galician trainera league')]
[('c', 'http://www.wikidata.org/entity/Q706023'), ('cname', '2004 UEFA European Under-21 Football Championship')]
[('c', 'http://www.wikidata.org/entity/Q8184272'), ('cname', 'Category:Tour de France stage winners')]
[('c', 'http://www.wikidata.org/entity/Q946600'), ('cname', '1996 World Cup of Hockey')]
[('c', 'http://www.wikidata.org/entity/Q127

50

In [11]:
#Q12
#same as before but for victory(P1346)
queryString = """
SELECT DISTINCT ?c ?cname
WHERE {
    ?a wdt:P1346 ?c.
       
    #labels
    ?c <http://schema.org/name> ?cname .
}
LIMIT 50
"""
print("Results")
run_query(queryString)

Results
[('c', 'http://www.wikidata.org/entity/Q1342082'), ('cname', 'Youssef Ziedan')]
[('c', 'http://www.wikidata.org/entity/Q8160652'), ('cname', 'Australia at the Hopman Cup')]
[('c', 'http://www.wikidata.org/entity/Q8206968'), ('cname', 'Hebraica y Macabi')]
[('c', 'http://www.wikidata.org/entity/Q8208548'), ('cname', 'Atlético Trujillo')]
[('c', 'http://www.wikidata.org/entity/Q10769440'), ('cname', 'Alena Mihulová')]
[('c', 'http://www.wikidata.org/entity/Q1202851'), ('cname', 'Kihachi Okamoto')]
[('c', 'http://www.wikidata.org/entity/Q1676812'), ('cname', 'Jabbar Patel')]
[('c', 'http://www.wikidata.org/entity/Q18788'), ('cname', 'Epik High')]
[('c', 'http://www.wikidata.org/entity/Q232273'), ('cname', 'Isabel Coixet')]
[('c', 'http://www.wikidata.org/entity/Q302645'), ('cname', 'Aaron Schneider')]
[('c', 'http://www.wikidata.org/entity/Q3080378'), ('cname', 'Yasuo Furuhata')]
[('c', 'http://www.wikidata.org/entity/Q3106353'), ('cname', 'Gilles Legrand')]
[('c', 'http://www.wik

50

<h3> TASK 1 ENDS HERE </h3>
In the following cells there are the BGP for winning and the one for participate in a competition. On top of that there is also the BGP for participate in an edition of Olympic Games.

In [14]:
#ANSWER TO 1): BGP for winning (P2522 or P1346)
queryString = """
SELECT DISTINCT ?a ?aname ?c ?cname
WHERE {
    ?a wdt:P106/wdt:P279 wd:Q2066131; #athlete
       wdt:P2522|wdt:P1346 ?c. #winning
       
    #labels
    ?a <http://schema.org/name> ?aname .
    ?c <http://schema.org/name> ?cname .
}
LIMIT 50
"""
print("Results")
run_query(queryString)

Results
[('aname', 'Alison Waters'), ('cname', "2014 Women's World Team Squash Championships")]
[('aname', 'Sarah-Jane Perry'), ('cname', "2014 Women's World Team Squash Championships")]
[('aname', 'Laura Massaro'), ('cname', "2014 Women's World Team Squash Championships")]
[('aname', 'Camille Serme'), ('cname', 'European Squash Individual Championships')]
[('aname', 'Raneem El Weleily'), ('cname', "Women's China Squash Open 2015")]
[('aname', 'Camille Serme'), ('cname', 'Monte-Carlo Squash Classic 2013')]
[('aname', 'Nicol David'), ('cname', 'Cayman Islands Open 2011')]
[('aname', 'Nour El Sherbini'), ('cname', "2016 Women's World Open Squash Championship")]
[('aname', 'Alison Waters'), ('cname', 'European Team Squash Championships 2017')]
[('aname', 'Sarah-Jane Perry'), ('cname', 'European Team Squash Championships 2017')]
[('aname', 'Nicol David'), ('cname', 'WSA World Series 2011')]
[('aname', 'Nicol David'), ('cname', "squash at the 2006 Asian Games – women's singles")]
[('aname',

50

In [15]:
#ANSWER TO 1): BGP for participating (P710 or P1344)
queryString = """
SELECT DISTINCT ?a ?aname ?c ?cname
WHERE {
    ?a wdt:P106/wdt:P279 wd:Q2066131; #athlete
       wdt:P710|wdt:P1344 ?c. #participate
       
    #labels
    ?a <http://schema.org/name> ?aname .
    ?c <http://schema.org/name> ?cname .
}
LIMIT 50
"""
print("Results")
run_query(queryString)

Results
[('a', 'http://www.wikidata.org/entity/Q50225176'), ('aname', 'Jesse Engelbrecht'), ('c', 'http://www.wikidata.org/entity/Q7120597'), ('cname', 'PSA World Tour 2012')]
[('a', 'http://www.wikidata.org/entity/Q7088030'), ('aname', 'Olivia Blatchford'), ('c', 'http://www.wikidata.org/entity/Q2049076'), ('cname', '2011 Pan American Games')]
[('a', 'http://www.wikidata.org/entity/Q592452'), ('aname', 'Miguel Ángel Rodríguez'), ('c', 'http://www.wikidata.org/entity/Q2049076'), ('cname', '2011 Pan American Games')]
[('a', 'http://www.wikidata.org/entity/Q6165464'), ('aname', 'Javier Castilla'), ('c', 'http://www.wikidata.org/entity/Q2049076'), ('cname', '2011 Pan American Games')]
[('a', 'http://www.wikidata.org/entity/Q2935075'), ('aname', 'Camille Serme'), ('c', 'http://www.wikidata.org/entity/Q3083264'), ('cname', 'World Games Wroclaw 2017')]
[('a', 'http://www.wikidata.org/entity/Q119692'), ('aname', 'Simon Rösner'), ('c', 'http://www.wikidata.org/entity/Q3083264'), ('cname', 'Wor

50

In [16]:
#Q13
#participating (P710 or P1344 ) limited to Olympic Games
queryString = """
SELECT DISTINCT ?a ?aname ?c ?cname
WHERE {
    ?a wdt:P106/wdt:P279 wd:Q2066131; #athlete
       wdt:P710|wdt:P1344 ?c. #participate
    ?c wdt:P31 wd:Q159821. #olympic games
       
    #labels
    ?a <http://schema.org/name> ?aname .
    ?c <http://schema.org/name> ?cname .
}
LIMIT 30
"""
print("Results")
run_query(queryString)

Results
[('a', 'http://www.wikidata.org/entity/Q16014208'), ('aname', 'Jan Vrolijk'), ('c', 'http://www.wikidata.org/entity/Q8150'), ('cname', '1936 Summer Olympics')]
[('a', 'http://www.wikidata.org/entity/Q665100'), ('aname', 'Frank Amyot'), ('c', 'http://www.wikidata.org/entity/Q8150'), ('cname', '1936 Summer Olympics')]
[('a', 'http://www.wikidata.org/entity/Q3362309'), ('aname', 'Erik Bladström'), ('c', 'http://www.wikidata.org/entity/Q8150'), ('cname', '1936 Summer Olympics')]
[('a', 'http://www.wikidata.org/entity/Q700944'), ('aname', 'Fritz Landertinger'), ('c', 'http://www.wikidata.org/entity/Q8150'), ('cname', '1936 Summer Olympics')]
[('a', 'http://www.wikidata.org/entity/Q5126665'), ('aname', 'Clarence McNutt'), ('c', 'http://www.wikidata.org/entity/Q8150'), ('cname', '1936 Summer Olympics')]
[('a', 'http://www.wikidata.org/entity/Q453677'), ('aname', 'Gregor Hradetzky'), ('c', 'http://www.wikidata.org/entity/Q8150'), ('cname', '1936 Summer Olympics')]
[('a', 'http://www.wi

30

<h3> TASK 2 STARTS HERE </h3>
Identify the BGP for different sports or types of competitions.

Using the result from task 1, i.e. the BGP for participating in Olympics, I am trying to find something related to sports or types of competitions.
Retrieve some instances of Olympics.

In [4]:
#Q13
#check  relationship with participate in olympic games
queryString = """
SELECT DISTINCT ?c ?cname
WHERE {
    ?a wdt:P106/wdt:P279 wd:Q2066131; #athlete
       wdt:P710|wdt:P1344 ?c. #participate
    ?c wdt:P31 wd:Q159821. #olympic games
    
    #labels
    ?c <http://schema.org/name> ?cname .
}
LIMIT 30
"""
print("Results")
run_query(queryString)

Results
[('c', 'http://www.wikidata.org/entity/Q8150'), ('cname', '1936 Summer Olympics')]
[('c', 'http://www.wikidata.org/entity/Q8098'), ('cname', '1904 Summer Olympics')]
[('c', 'http://www.wikidata.org/entity/Q8415'), ('cname', '1960 Summer Olympics')]
[('c', 'http://www.wikidata.org/entity/Q8438'), ('cname', '1972 Summer Olympics')]
[('c', 'http://www.wikidata.org/entity/Q181278'), ('cname', '2020 Summer Olympics')]
[('c', 'http://www.wikidata.org/entity/Q8088'), ('cname', '1900 Summer Olympics')]
[('c', 'http://www.wikidata.org/entity/Q8444'), ('cname', '1976 Summer Olympics')]
[('c', 'http://www.wikidata.org/entity/Q8470'), ('cname', '1988 Summer Olympics')]
[('c', 'http://www.wikidata.org/entity/Q8488'), ('cname', '1992 Summer Olympics')]
[('c', 'http://www.wikidata.org/entity/Q8531'), ('cname', '1996 Summer Olympics')]
[('c', 'http://www.wikidata.org/entity/Q8544'), ('cname', '2000 Summer Olympics')]
[('c', 'http://www.wikidata.org/entity/Q8558'), ('cname', '2004 Summer Olympi

30

In [3]:
#Q14
#same as above but using winning properties to connect to olympic games
queryString = """
SELECT DISTINCT ?c ?cname
WHERE {
    ?a wdt:P106/wdt:P279 wd:Q2066131; #athlete
       wdt:P2522|wdt:P1346 ?c. #winning
    ?c wdt:P31 wd:Q159821. #olympic games
       
    #labels
    ?c <http://schema.org/name> ?cname .
}
LIMIT 50
"""
print("Results")
run_query(queryString)

Results
[('c', 'http://www.wikidata.org/entity/Q8567'), ('cname', '2008 Summer Olympics')]
[('c', 'http://www.wikidata.org/entity/Q181278'), ('cname', '2020 Summer Olympics')]
[('c', 'http://www.wikidata.org/entity/Q8438'), ('cname', '1972 Summer Olympics')]


3

In [5]:
#Q14
#change athlete to swimmer to find references to different type/sports 
queryString = """
SELECT DISTINCT ?c ?cname
WHERE {
    ?a wdt:P106/wdt:P279 wd:Q10843402; #swimmer
       wdt:P710|wdt:P1344 ?c. #participation
    ?c wdt:P31 wd:Q159821. #olympic games
       
    #labels
    ?c <http://schema.org/name> ?cname .
}
LIMIT 50
"""
print("Results")
run_query(queryString)

Results
[('c', 'http://www.wikidata.org/entity/Q8150'), ('cname', '1936 Summer Olympics')]
[('c', 'http://www.wikidata.org/entity/Q8415'), ('cname', '1960 Summer Olympics')]
[('c', 'http://www.wikidata.org/entity/Q8438'), ('cname', '1972 Summer Olympics')]
[('c', 'http://www.wikidata.org/entity/Q8444'), ('cname', '1976 Summer Olympics')]
[('c', 'http://www.wikidata.org/entity/Q8470'), ('cname', '1988 Summer Olympics')]
[('c', 'http://www.wikidata.org/entity/Q8488'), ('cname', '1992 Summer Olympics')]
[('c', 'http://www.wikidata.org/entity/Q8531'), ('cname', '1996 Summer Olympics')]
[('c', 'http://www.wikidata.org/entity/Q8544'), ('cname', '2000 Summer Olympics')]
[('c', 'http://www.wikidata.org/entity/Q8558'), ('cname', '2004 Summer Olympics')]
[('c', 'http://www.wikidata.org/entity/Q8567'), ('cname', '2008 Summer Olympics')]
[('c', 'http://www.wikidata.org/entity/Q8577'), ('cname', '2012 Summer Olympics')]
[('c', 'http://www.wikidata.org/entity/Q8613'), ('cname', '2016 Summer Olympics

16

In [6]:
#Q15
#same as above but without connection to olympic games
queryString = """
SELECT DISTINCT ?c ?cname
WHERE {
    ?a wdt:P106/wdt:P279 wd:Q10843402; #swimmer
       wdt:P710|wdt:P1344 ?c. #participation
       
    #labels
    ?c <http://schema.org/name> ?cname .
}
LIMIT 50
"""
print("Results")
run_query(queryString)

Results
[('c', 'http://www.wikidata.org/entity/Q1978175'), ('cname', '21st Congress of the Communist Party of the Soviet Union')]
[('c', 'http://www.wikidata.org/entity/Q8150'), ('cname', '1936 Summer Olympics')]
[('c', 'http://www.wikidata.org/entity/Q8415'), ('cname', '1960 Summer Olympics')]
[('c', 'http://www.wikidata.org/entity/Q8438'), ('cname', '1972 Summer Olympics')]
[('c', 'http://www.wikidata.org/entity/Q8444'), ('cname', '1976 Summer Olympics')]
[('c', 'http://www.wikidata.org/entity/Q8488'), ('cname', '1992 Summer Olympics')]
[('c', 'http://www.wikidata.org/entity/Q8531'), ('cname', '1996 Summer Olympics')]
[('c', 'http://www.wikidata.org/entity/Q8544'), ('cname', '2000 Summer Olympics')]
[('c', 'http://www.wikidata.org/entity/Q8558'), ('cname', '2004 Summer Olympics')]
[('c', 'http://www.wikidata.org/entity/Q8132'), ('cname', '1924 Summer Olympics')]
[('c', 'http://www.wikidata.org/entity/Q8420'), ('cname', '1964 Summer Olympics')]
[('c', 'http://www.wikidata.org/entity/Q

50

Retrieve some predicates and objects of 2000 Olympics.

In [10]:
#Q16
#closer look to Q8544 found above
queryString = """
SELECT DISTINCT ?p ?pname ?o ?oname
WHERE {
    
    wd:Q8544 ?p ?o .#2000 olympics
    
    #labels
    ?p <http://schema.org/name> ?pname .
    ?o <http://schema.org/name> ?oname .
}
LIMIT 50
"""
print("Results")
run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P527'), ('pname', 'has part'), ('o', 'http://www.wikidata.org/entity/Q513725'), ('oname', 'synchronized swimming at the 2000 Summer Olympics')]
[('p', 'http://www.wikidata.org/prop/direct/P527'), ('pname', 'has part'), ('o', 'http://www.wikidata.org/entity/Q579844'), ('oname', 'fencing at the 2000 Summer Olympics')]
[('p', 'http://www.wikidata.org/prop/direct/P527'), ('pname', 'has part'), ('o', 'http://www.wikidata.org/entity/Q847738'), ('oname', 'cycling at the 2000 Summer Olympics')]
[('p', 'http://www.wikidata.org/prop/direct/P527'), ('pname', 'has part'), ('o', 'http://www.wikidata.org/entity/Q679703'), ('oname', 'judo at the 2000 Summer Olympics')]
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('pname', 'instance of'), ('o', 'http://www.wikidata.org/entity/Q159821'), ('oname', 'Summer Olympic Games')]
[('p', 'http://www.wikidata.org/prop/direct/P2670'), ('pname', 'has parts of the class'), ('o', 'http://www.wikidata.org/ent

50

Try to retrieve some olympic competitions using P527.

In [11]:
#Q17
#closer look to Q8544 + property P527
queryString = """
SELECT DISTINCT ?o ?oname
WHERE {
    
    wd:Q8544 wdt:P527 ?o .#2000 olympics
    
    #labels
    ?o <http://schema.org/name> ?oname .
}
LIMIT 50
"""
print("Results")
run_query(queryString)

Results
[('o', 'http://www.wikidata.org/entity/Q513725'), ('oname', 'synchronized swimming at the 2000 Summer Olympics')]
[('o', 'http://www.wikidata.org/entity/Q579844'), ('oname', 'fencing at the 2000 Summer Olympics')]
[('o', 'http://www.wikidata.org/entity/Q847738'), ('oname', 'cycling at the 2000 Summer Olympics')]
[('o', 'http://www.wikidata.org/entity/Q679703'), ('oname', 'judo at the 2000 Summer Olympics')]
[('o', 'http://www.wikidata.org/entity/Q754594'), ('oname', 'wrestling at the 2000 Summer Olympics')]
[('o', 'http://www.wikidata.org/entity/Q843711'), ('oname', 'equestrian at the 2000 Summer Olympics')]
[('o', 'http://www.wikidata.org/entity/Q855408'), ('oname', 'canoeing at the 2000 Summer Olympics')]
[('o', 'http://www.wikidata.org/entity/Q570002'), ('oname', 'table tennis at the 2000 Summer Olympics')]
[('o', 'http://www.wikidata.org/entity/Q843708'), ('oname', 'shooting at the 2000 Summer Olympics')]
[('o', 'http://www.wikidata.org/entity/Q661494'), ('oname', 'tennis a

36

Retrieve all editions of Olympic Games.

In [12]:
#Q18
#P527 on 'general' olympics entity
queryString = """
SELECT DISTINCT ?o ?oname
WHERE {
    
    wd:Q159821 wdt:P527 ?o .#olympics
    
    #labels
    ?o <http://schema.org/name> ?oname .
}
LIMIT 50
"""
print("Results")
run_query(queryString)

Results
[('o', 'http://www.wikidata.org/entity/Q8098'), ('oname', '1904 Summer Olympics')]
[('o', 'http://www.wikidata.org/entity/Q8150'), ('oname', '1936 Summer Olympics')]
[('o', 'http://www.wikidata.org/entity/Q8415'), ('oname', '1960 Summer Olympics')]
[('o', 'http://www.wikidata.org/entity/Q181278'), ('oname', '2020 Summer Olympics')]
[('o', 'http://www.wikidata.org/entity/Q8088'), ('oname', '1900 Summer Olympics')]
[('o', 'http://www.wikidata.org/entity/Q8438'), ('oname', '1972 Summer Olympics')]
[('o', 'http://www.wikidata.org/entity/Q8444'), ('oname', '1976 Summer Olympics')]
[('o', 'http://www.wikidata.org/entity/Q8470'), ('oname', '1988 Summer Olympics')]
[('o', 'http://www.wikidata.org/entity/Q8488'), ('oname', '1992 Summer Olympics')]
[('o', 'http://www.wikidata.org/entity/Q8531'), ('oname', '1996 Summer Olympics')]
[('o', 'http://www.wikidata.org/entity/Q8544'), ('oname', '2000 Summer Olympics')]
[('o', 'http://www.wikidata.org/entity/Q8558'), ('oname', '2004 Summer Olympi

36

By iterating two times P527 I could be able to list all the competitions included in all the editions of Olympics.

In [13]:
#Q18
#iterate P527 to get the sport entities related to all olympics editions
queryString = """
SELECT DISTINCT ?o ?oname
WHERE {
    
    wd:Q159821 wdt:P527/wdt:P527 ?o .#olympics
    
    #labels
    ?o <http://schema.org/name> ?oname .
}
LIMIT 50
"""
print("Results")
run_query(queryString)

Results
[('o', 'http://www.wikidata.org/entity/Q765662'), ('oname', 'water motorsports at the 1908 Summer Olympics')]
[('o', 'http://www.wikidata.org/entity/Q221132'), ('oname', 'field hockey at the 2008 Summer Olympics')]
[('o', 'http://www.wikidata.org/entity/Q267154'), ('oname', 'diving at the 1932 Summer Olympics')]
[('o', 'http://www.wikidata.org/entity/Q369891'), ('oname', 'diving at the 1984 Summer Olympics')]
[('o', 'http://www.wikidata.org/entity/Q770656'), ('oname', 'diving at the 1920 Summer Olympics')]
[('o', 'http://www.wikidata.org/entity/Q7427963'), ('oname', 'savate at the 1924 Summer Olympics')]
[('o', 'http://www.wikidata.org/entity/Q17063692'), ('oname', 'wheelchair racing at the 1988 Summer Olympics')]
[('o', 'http://www.wikidata.org/entity/Q17066430'), ('oname', 'wheelchair racing at the 1984 Summer Olympics')]
[('o', 'http://www.wikidata.org/entity/Q599584'), ('oname', 'equestrian at the 1996 Summer Olympics')]
[('o', 'http://www.wikidata.org/entity/Q283048'), ('o

50

Let's have a closer look to 1964 Olympics.

In [14]:
#Q19
#closer look to Q202510 (1964 Olympics)
queryString = """
SELECT DISTINCT ?p ?pname ?o ?oname
WHERE {
    
    wd:Q202510 ?p ?o .#football in 1964 olympic games
    
    #labels
    ?p <http://schema.org/name> ?pname .
    ?o <http://schema.org/name> ?oname .
}
LIMIT 50
"""
print("Results")
run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P17'), ('pname', 'country'), ('o', 'http://www.wikidata.org/entity/Q17'), ('oname', 'Japan')]
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('pname', 'instance of'), ('o', 'http://www.wikidata.org/entity/Q26132862'), ('oname', 'Olympic sports discipline event')]
[('p', 'http://www.wikidata.org/prop/direct/P641'), ('pname', 'sport'), ('o', 'http://www.wikidata.org/entity/Q2736'), ('oname', 'association football')]
[('p', 'http://www.wikidata.org/prop/direct/P361'), ('pname', 'part of'), ('o', 'http://www.wikidata.org/entity/Q8420'), ('oname', '1964 Summer Olympics')]
[('p', 'http://www.wikidata.org/prop/direct/P276'), ('pname', 'location'), ('o', 'http://www.wikidata.org/entity/Q330033'), ('oname', 'National Stadium')]
[('p', 'http://www.wikidata.org/prop/direct/P131'), ('pname', 'located in the administrative territorial entity'), ('o', 'http://www.wikidata.org/entity/Q7473516'), ('oname', 'Tokyo')]
[('p', 'http://www.wikidata.or

10

<h3> TASK 2 ENDS HERE </h3>
In the following cell there is the BGP for identify a sport (using P641) or different types of competitions.

In [16]:
#ANSWER TO 2) : BGP for different sports
#for each athlete, this query retrieves the sport (P641 found above) of the competiton he/she took part in
queryString = """
SELECT DISTINCT ?o ?oname
WHERE {
    ?a wdt:P106/wdt:P279 wd:Q2066131; #athlete
       wdt:P710|wdt:P1344 ?c. #participate
    ?c wdt:P641 ?o.
       
    #labels
    ?o <http://schema.org/name> ?oname .
}
LIMIT 50
"""
print("Results")
run_query(queryString)

Results
[('o', 'http://www.wikidata.org/entity/Q327223'), ('oname', 'freestyle wrestling')]
[('o', 'http://www.wikidata.org/entity/Q3312129'), ('oname', 'track and field')]
[('o', 'http://www.wikidata.org/entity/Q1455'), ('oname', 'field hockey')]
[('o', 'http://www.wikidata.org/entity/Q167170'), ('oname', 'multi-sport event')]
[('o', 'http://www.wikidata.org/entity/Q1734'), ('oname', 'volleyball')]
[('o', 'http://www.wikidata.org/entity/Q179687'), ('oname', 'cross-country skiing')]
[('o', 'http://www.wikidata.org/entity/Q204686'), ('oname', 'winter sport')]
[('o', 'http://www.wikidata.org/entity/Q206989'), ('oname', 'shooting sport')]
[('o', 'http://www.wikidata.org/entity/Q2736'), ('oname', 'association football')]
[('o', 'http://www.wikidata.org/entity/Q31920'), ('oname', 'swimming')]
[('o', 'http://www.wikidata.org/entity/Q41466'), ('oname', 'ice hockey')]
[('o', 'http://www.wikidata.org/entity/Q5369'), ('oname', 'baseball')]
[('o', 'http://www.wikidata.org/entity/Q5372'), ('oname'

50

<h3>TASK 3 STARTS HERE</h3>
Is there some French or German Athlete participating to any olympic event with double citizenship?

Retrieve the french athletes who played for other country using P1532 (country for sport) found in the first query.

In [19]:
#Q20
#french athletes who play for other countries
#in the first query we found (P1532)= country for sport
queryString = """
SELECT DISTINCT ?a ?aname
WHERE {
    ?a wdt:P106/wdt:P279 wd:Q2066131; #athlete
       wdt:P27 wd:Q142; #french
       wdt:P710|wdt:P1344 ?o; #participating in competition
       wdt:P1532 ?country.
    
    ?o wdt:P31 wd:Q159821. #olympic games
       
    FILTER(?country != wd:Q142) #french but playing for other country
    
    #labels
    ?a <http://schema.org/name> ?aname .
}
LIMIT 50
"""
print("Results")
run_query(queryString)

Results
[('a', 'http://www.wikidata.org/entity/Q536340'), ('aname', 'Angelo Parisi')]
[('a', 'http://www.wikidata.org/entity/Q3271440'), ('aname', 'Léon de Lunden')]
[('a', 'http://www.wikidata.org/entity/Q348493'), ('aname', 'Jessica Fox')]
[('a', 'http://www.wikidata.org/entity/Q83853'), ('aname', 'Henryk Kasperczak')]
[('a', 'http://www.wikidata.org/entity/Q3358507'), ('aname', 'Ousmane Diarra')]
[('a', 'http://www.wikidata.org/entity/Q1905564'), ('aname', 'Martine Oppliger')]
[('a', 'http://www.wikidata.org/entity/Q3196017'), ('aname', 'Khalid Rahilou')]
[('a', 'http://www.wikidata.org/entity/Q513557'), ('aname', 'Mickaël Conjungo')]
[('a', 'http://www.wikidata.org/entity/Q229085'), ('aname', 'Françoise Mbango Etone')]
[('a', 'http://www.wikidata.org/entity/Q216557'), ('aname', 'Mohamed Sissoko')]
[('a', 'http://www.wikidata.org/entity/Q1644734'), ('aname', 'Hajnalka Kiraly')]
[('a', 'http://www.wikidata.org/entity/Q925242'), ('aname', 'Jamal Alioui')]
[('a', 'http://www.wikidata.o

28

Let's have a look at Tony Parker.

In [20]:
#Q21
#closer look to Tony Parker  (Q193108)
queryString = """
SELECT DISTINCT ?oname ?cname
WHERE {
    wd:Q193108 wdt:P710|wdt:P1344 ?o; #participating in competition
               wdt:P1532 ?country.
    
    ?o wdt:P31 wd:Q159821. #olympic games
    
    #labels
    ?o <http://schema.org/name> ?oname .
    ?country <http://schema.org/name> ?cname .
}
LIMIT 50
"""
print("Results")
run_query(queryString)

Results
[('oname', '2012 Summer Olympics'), ('cname', 'United States of America')]
[('oname', '2016 Summer Olympics'), ('cname', 'United States of America')]


2

The previous result is odd since he never played for USA. I'll go deeper to find why.

In [23]:
#Q22
#again because previous results are weird (Q193108)
queryString = """
SELECT DISTINCT ?pname ?oname
WHERE {
    wd:Q193108 ?p ?o.
    
    #labels
    ?o <http://schema.org/name> ?oname .
    ?p <http://schema.org/name> ?pname .
}
LIMIT 50
"""
print("Results")
run_query(queryString)

Results
[('pname', 'occupation'), ('oname', 'radio personality')]
[('pname', 'occupation'), ('oname', 'basketball player')]
[('pname', 'employer'), ('oname', 'San Antonio Spurs')]
[('pname', 'league'), ('oname', 'National Basketball Association')]
[('pname', 'participant in'), ('oname', 'FIBA EuroBasket 2011')]
[('pname', 'participant in'), ('oname', 'EuroBasket 2005')]
[('pname', 'participant in'), ('oname', '2012 Summer Olympics')]
[('pname', 'participant in'), ('oname', '2016 Summer Olympics')]
[('pname', 'participant in'), ('oname', 'FIBA EuroBasket 2015')]
[('pname', 'participant in'), ('oname', 'EuroBasket 2007')]
[('pname', 'participant in'), ('oname', 'FIBA EuroBasket 2013')]
[('pname', 'participant in'), ('oname', 'FIBA EuroBasket 2009')]
[('pname', 'participant in'), ('oname', 'EuroBasket 2003')]
[('pname', 'participant in'), ('oname', 'EuroBasket 2001')]
[('pname', 'named after'), ('oname', 'Tony Parker, Sr.')]
[('pname', 'languages spoken, written or signed'), ('oname', 'Fr

43

Try to run the same query but for another athlete with multiple citizenships.

In [24]:
#Q23
#closer look to Aubameyang (Q44977)
queryString = """
SELECT DISTINCT ?country ?cname
WHERE {
    wd:Q44977 wdt:P27 ?country .
    
    #labels
    ?country <http://schema.org/name> ?cname .
}
LIMIT 50
"""
print("Results")
run_query(queryString)

Results
[('country', 'http://www.wikidata.org/entity/Q1000'), ('cname', 'Gabon')]
[('country', 'http://www.wikidata.org/entity/Q142'), ('cname', 'France')]
[('country', 'http://www.wikidata.org/entity/Q29'), ('cname', 'Spain')]


3

Retrieve some french or german athletes with double citizenship who took part in olympics.

In [30]:
#Q24
#retrieve french or german athletes with double citizenship who took part in olympics
queryString = """
SELECT ?athlete
WHERE {
    ?athlete wdt:P710|wdt:P1344 ?o; #participating in competition
             wdt:P27 ?country;
             wdt:P1532 ?cs. #country for sport
    
    ?o wdt:P31 wd:Q159821. #olympic games
    FILTER( (?cs = wd:Q142) || (?cs=wd:Q183) ) 
}
GROUP BY (?athlete)
HAVING (COUNT(?country)>1)
LIMIT 10
"""
print("Results")
run_query(queryString)

Results
[('athlete', 'http://www.wikidata.org/entity/Q435068')]
[('athlete', 'http://www.wikidata.org/entity/Q26262098')]
[('athlete', 'http://www.wikidata.org/entity/Q4261')]
[('athlete', 'http://www.wikidata.org/entity/Q440839')]
[('athlete', 'http://www.wikidata.org/entity/Q567084')]
[('athlete', 'http://www.wikidata.org/entity/Q65790')]
[('athlete', 'http://www.wikidata.org/entity/Q44781')]
[('athlete', 'http://www.wikidata.org/entity/Q13485')]
[('athlete', 'http://www.wikidata.org/entity/Q609877')]
[('athlete', 'http://www.wikidata.org/entity/Q452740')]


10

Count how many athlete have more than one citizenships and took part in Olympics representing France.

In [8]:
#Q25
#count how many for France
queryString = """
SELECT (COUNT(?athlete) AS ?french)
{
SELECT ?athlete (COUNT (?country) AS ?nCountries)
WHERE {
    ?athlete wdt:P710|wdt:P1344 ?o; #participating in competition
             wdt:P27 ?country;
             wdt:P1532 ?cs. #country for sport
    
    ?o wdt:P31 wd:Q159821. #olympic games
    FILTER (?cs = wd:Q142) 
}
GROUP BY (?athlete)
HAVING (COUNT(?country)>1)
}
"""
print("Results")
run_query(queryString)

Results
[('french', '66')]


1

66 seems low, so in the next cells I want to check if there are better way to connect athletes and Olympic Games.

In [6]:
#Q26
#66 seems pretty low, check if there are a better way to connect olympics and competition
#try to use P361 (part of) found in Q19
queryString = """
SELECT ?o ?oname
WHERE {

    ?o wdt:P361 wd:Q159821. #olympic games
    #Label
    ?oname <http://schema.org/name> ?oname .
   
}
LIMIT 30
"""
print("Results")
run_query(queryString)

Results
The operation failed timed out


In [5]:
#Q27
#66 seems pretty low, check if there are a better way to connect olympics and competition
#no result, so try to iterate over P361 (part of)
queryString = """
SELECT ?o ?oname
WHERE {

    ?o wdt:P361/wdt:P361 wd:Q159821. #olympic games
    #Label
    ?oname <http://schema.org/name> ?oname .
   
}
LIMIT 30
"""
print("Results")
run_query(queryString)

Results
Empty


0

In [4]:
#Q28
#66 seems pretty low, check if there are a better way to connect olympics and competition
#keep P31 and use Q26132862 (olympics sport event)

queryString = """
SELECT ?athlete
WHERE {
    ?athlete wdt:P710|wdt:P1344 ?o; #participating in competition
             wdt:P27 ?country;
             wdt:P1532 ?cs. #country for sport
    
    ?o wdt:P31 wd:Q26132862. #olympic sport event
    FILTER (?cs = wd:Q142) 
}
GROUP BY (?athlete)
HAVING (COUNT(?country)>1)
LIMIT 10
"""
print("Results")
run_query(queryString)

Results
[('athlete', 'http://www.wikidata.org/entity/Q3195752')]
[('athlete', 'http://www.wikidata.org/entity/Q229831')]
[('athlete', 'http://www.wikidata.org/entity/Q180872')]
[('athlete', 'http://www.wikidata.org/entity/Q232633')]
[('athlete', 'http://www.wikidata.org/entity/Q229002')]
[('athlete', 'http://www.wikidata.org/entity/Q242959')]
[('athlete', 'http://www.wikidata.org/entity/Q28325')]
[('athlete', 'http://www.wikidata.org/entity/Q192661')]


8

In [3]:
#Q29
#closer look to an olympic sport (Q160007 athletics at 1972 olympics)
queryString = """
SELECT ?p ?pname ?o ?oname
WHERE {

    wd:Q160007 ?p ?o.
    FILTER(!isLiteral(?o))
    
    #Label
    ?pname <http://schema.org/name> ?pname .
    ?oname <http://schema.org/name> ?oname .
}
LIMIT 30
"""
print("Results")
run_query(queryString)

Results
The operation failed EndPointInternalError: endpoint returned code 500 and response. 

Response:
b'Virtuoso 42000 Error The estimated execution time 300931360 (sec) exceeds the limit of 800 (sec).\n\nSPARQL query:\n\n##-567b0d5169-##\nPREFIX wd: <http://www.wikidata.org/entity/> \nPREFIX wdt: <http://www.wikidata.org/prop/direct/> \nPREFIX sc: <http://schema.org/>\n\n\nSELECT ?p ?pname ?o ?oname\nWHERE {\n\n    wd:Q160007 ?p ?o.\n    FILTER(!isLiteral(?o))\n    \n    #Label\n    ?pname <http://schema.org/name> ?pname .\n    ?oname <http://schema.org/name> ?oname .\n}\nLIMIT 30\n'


In [5]:
#Q30
#closer look to an olympic sport (Q766357 athletics at some olympics)
queryString = """
SELECT ?p ?o ?oname
WHERE {

    wd:Q766357 ?p ?o.
    FILTER(!isLiteral(?o))
    
    #Label
    ?oname <http://schema.org/name> ?oname .
}
LIMIT 30
"""
print("Results")
run_query(queryString)

Results
The operation failed timed out


In [7]:
#Q31
#closer look to an olympic sport (Q766357 athletics at some olympics)
queryString = """
SELECT ?o ?oname
WHERE {

    wd:Q766357 wdt:P527 ?o.
    FILTER(!isLiteral(?o))
    
    #Label
    ?oname <http://schema.org/name> ?oname .
}
LIMIT 30
"""
print("Results")
run_query(queryString)

Results
The operation failed timed out


In [9]:
#Q32
#last try
#closer look to an olympic sport (Q581196 cycling at XXVI olympics)
queryString = """
SELECT ?o ?oname
WHERE {

    wd:Q581196 wdt:P527 ?o.
    FILTER(!isLiteral(?o))
    
    #Label
    ?oname <http://schema.org/name> ?oname .
}
LIMIT 30
"""
print("Results")
run_query(queryString)

Results
Empty


0

In [10]:
#Q33
#check the reverse path: from a specific athlete (Q435068) to olympic games to see if there are other ways to connect the nodes
queryString = """
SELECT ?o ?oname
WHERE {

    wd:Q435068 wdt:P1344|wdt:710 ?o.
    ?o wdt:P31 wd:Q159821. #olympic games
    
    #Label
    ?oname <http://schema.org/name> ?oname .
}
LIMIT 30
"""
print("Results")
run_query(queryString)

Results
Empty


0

In [11]:
#Q34
#check the reverse path: from generic french athlete to olympic games to see if there are other ways to connect the nodes
queryString = """
SELECT ?p ?pname
WHERE {

    ?a wdt:P106/wdt:P279 wd:Q2066131. 
    ?o ?p wd:Q159821. #olympic games
    
    #Label
    ?pname <http://schema.org/name> ?pname .
}
LIMIT 30
"""
print("Results")
run_query(queryString)

Results
The operation failed EndPointInternalError: endpoint returned code 500 and response. 

Response:
b'Virtuoso 42000 Error The estimated execution time 8483 (sec) exceeds the limit of 800 (sec).\n\nSPARQL query:\n\n##-567b0d5169-##\nPREFIX wd: <http://www.wikidata.org/entity/> \nPREFIX wdt: <http://www.wikidata.org/prop/direct/> \nPREFIX sc: <http://schema.org/>\n\n\nSELECT ?p ?pname\nWHERE {\n\n    ?a wdt:P106/wdt:P279 wd:Q2066131. \n    ?o ?p wd:Q159821. #olympic games\n    \n    #Label\n    ?pname <http://schema.org/name> ?pname .\n}\nLIMIT 30\n'


In [13]:
#Q35
#try to use part of (P361) found in Q19
queryString = """
SELECT ?athlete
WHERE {
    ?athlete wdt:P710|wdt:P1344 ?o; #participating in competition
             wdt:P27 ?country;
             wdt:P1532 ?cs. #country for sport
             
    ?o wdt:P361 wd:Q159821. #olympic games
    FILTER( (?cs = wd:Q142) || (?cs=wd:Q183) ) 
}
GROUP BY (?athlete)
HAVING (COUNT(?country)>1)
LIMIT 10
"""
print("Results")
run_query(queryString)

Results
Empty


0

<h3> TASK 3 ENDS HERE </h3>
In the next cell there is the BGP to find the french or german athlete who took part in olympics and have double citizenship.
I used BGP that connect athlete and participating in olympics found in task 2 and added the property P1532 which shows the nation an athlete compete for. The first result seems low so i tried other ways to connect athlete and olympics partecipation but unsuccessfully.

In [14]:
#ANSWER TO TASK 3)
#this is the BGP to find the french or german athlete who took part in olympics and have double citizenship
#i used BGP that connect athlete and participating in olympics found in task 2 and added the property P1532 which shows the
#nation an athlete compete for
#the first result seems low so i tried other ways to connect athlete and olympics partecipation but unsuccessfully
queryString = """
SELECT ?athlete
WHERE {
    ?athlete wdt:P710|wdt:P1344 ?o; #participating in competition
             wdt:P27 ?country;
             wdt:P1532 ?cs. #country for sport
    
    ?o wdt:P31 wd:Q159821. #olympic games
    FILTER( (?cs = wd:Q142) || (?cs=wd:Q183) ) 
}
GROUP BY (?athlete)
HAVING (COUNT(?country)>1)
LIMIT 10
"""
print("Results")
run_query(queryString)

Results


Count how many athlete participated in Olympic Games for Germany and also have double citizenship.

In [15]:
#ADDITIONAL ON TASK 3

#France has 66 olympic athletes with double citizenships 
#count how many for Germany
queryString = """
SELECT (COUNT(?athlete) AS ?german)
{
SELECT ?athlete (COUNT (?country) AS ?nCountries)
WHERE {
    ?athlete wdt:P710|wdt:P1344 ?o; #participating in competition
             wdt:P27 ?country;
             wdt:P1532 ?cs. #country for sport
    
    ?o wdt:P31 wd:Q159821. #olympic games
    FILTER (?cs = wd:Q183) 
}
GROUP BY (?athlete)
HAVING (COUNT(?country)>1)
}
"""
print("Results")
run_query(queryString)

Results
[('german', '115')]


1

Check if an athlete from France or Gemrany with double citizenship won an olympic competition.

In [16]:
#ADDITIONAL ON TASK 3

#France has 66 olympic athletes with double citizenships, Germany 115
#check if someone won something related to olympic games
queryString = """
SELECT ?athlete
WHERE {
    ?athlete wdt:P2522|wdt:P1346 ?o; #winning in competition
             wdt:P27 ?country;
             wdt:P1532 ?cs. #country for sport
    
    ?o wdt:P31 wd:Q159821. #olympic games
    FILTER( (?cs = wd:Q142) || (?cs=wd:Q183) ) 
}
GROUP BY (?athlete)
HAVING (COUNT(?country)>1)
LIMIT 10
"""
print("Results")
run_query(queryString)

Results
Empty


0

On top of TASK 3, I compared how many german athletes have double citizenships and I found out that they are 115, so about the double with respect to
France. Also an athlete with double citizenship from France or German never won anything related to Olympic Games.

<h3> TASK 4 STARTS HERE </h3>
Compare number of participations and winning events across different dimensions
4.1 In which specific sport France has more winners than Germany
4.2 Who has the highest number of female winners
4.3 For each country , which are the atheletes with the highest number of participations

Using TASK 1 result, try to find a BGP for winning a competition in Olympics.

In [3]:
#Q36
#find a path between athletes and win in Olympic Games using TASK 1 results
queryString = """
SELECT DISTINCT ?a ?aname ?c ?cname
WHERE {
    ?a wdt:P106/wdt:P279 wd:Q2066131; #athlete
       wdt:P2522|wdt:P1346 ?c. #winning
    ?c wdt:P31 wd:Q159821 . #instance of olympics
       
    #labels
    ?a <http://schema.org/name> ?aname .
    ?c <http://schema.org/name> ?cname .
}
LIMIT 50
"""
print("Results")
run_query(queryString)

Results
[('a', 'http://www.wikidata.org/entity/Q1174439'), ('aname', 'David Florence'), ('c', 'http://www.wikidata.org/entity/Q8567'), ('cname', '2008 Summer Olympics')]
[('a', 'http://www.wikidata.org/entity/Q1189365'), ('aname', 'Dmitriy Dobroskok'), ('c', 'http://www.wikidata.org/entity/Q8567'), ('cname', '2008 Summer Olympics')]
[('a', 'http://www.wikidata.org/entity/Q11156528'), ('aname', 'Matthew Wells'), ('c', 'http://www.wikidata.org/entity/Q8567'), ('cname', '2008 Summer Olympics')]
[('a', 'http://www.wikidata.org/entity/Q127132'), ('aname', 'Paddy Barnes'), ('c', 'http://www.wikidata.org/entity/Q8567'), ('cname', '2008 Summer Olympics')]
[('a', 'http://www.wikidata.org/entity/Q1048599'), ('aname', 'Yordenis Ugás'), ('c', 'http://www.wikidata.org/entity/Q8567'), ('cname', '2008 Summer Olympics')]
[('a', 'http://www.wikidata.org/entity/Q1165461'), ('aname', 'Daouda Sow'), ('c', 'http://www.wikidata.org/entity/Q8567'), ('cname', '2008 Summer Olympics')]
[('a', 'http://www.wikida

42

In the last resultset there are very few results.

In [6]:
#Q37
#previous query retrived very few athletes (and also all about the same edition)
#find new link between competition and olympics
queryString = """
SELECT DISTINCT ?p ?pname
WHERE {
    ?a wdt:P106/wdt:P279 wd:Q2066131; #athlete
       wdt:P2522|wdt:P1346 ?c. #winning
    ?c ?p wd:Q159821 . #instance of olympics
       
    #labels
    ?p <http://schema.org/name> ?pname .
}
LIMIT 50
"""
print("Results")
run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('pname', 'instance of')]
[('p', 'http://www.wikidata.org/prop/direct/P2670'), ('pname', 'has parts of the class')]


2

Explore a competition belonging to Olympic Games.

In [7]:
#Q38
#try to have a closer look to olympic game competition using Q19 results
queryString = """
SELECT DISTINCT ?c ?cname
WHERE {
    
    wd:Q8544 wdt:P527 ?c.

    #labels
    ?c <http://schema.org/name> ?cname .
}
LIMIT 10
"""
print("Results")
run_query(queryString)

Results
[('c', 'http://www.wikidata.org/entity/Q513725'), ('cname', 'synchronized swimming at the 2000 Summer Olympics')]
[('c', 'http://www.wikidata.org/entity/Q579844'), ('cname', 'fencing at the 2000 Summer Olympics')]
[('c', 'http://www.wikidata.org/entity/Q847738'), ('cname', 'cycling at the 2000 Summer Olympics')]
[('c', 'http://www.wikidata.org/entity/Q679703'), ('cname', 'judo at the 2000 Summer Olympics')]
[('c', 'http://www.wikidata.org/entity/Q754594'), ('cname', 'wrestling at the 2000 Summer Olympics')]
[('c', 'http://www.wikidata.org/entity/Q843711'), ('cname', 'equestrian at the 2000 Summer Olympics')]
[('c', 'http://www.wikidata.org/entity/Q855408'), ('cname', 'canoeing at the 2000 Summer Olympics')]
[('c', 'http://www.wikidata.org/entity/Q570002'), ('cname', 'table tennis at the 2000 Summer Olympics')]
[('c', 'http://www.wikidata.org/entity/Q843708'), ('cname', 'shooting at the 2000 Summer Olympics')]
[('c', 'http://www.wikidata.org/entity/Q661494'), ('cname', 'tennis a

10

In [8]:
#Q39
#try to have a closer look to olympic game competition using Q19 results
queryString = """
SELECT DISTINCT ?c ?cname
WHERE {
    
    wd:Q8544 wdt:P527/wdt:P527 ?c.

    #labels
    ?c <http://schema.org/name> ?cname .
}
LIMIT 10
"""
print("Results")
run_query(queryString)

Results
[('c', 'http://www.wikidata.org/entity/Q3628783'), ('cname', "athletics at the 2000 Summer Olympics – men's pole vault")]
[('c', 'http://www.wikidata.org/entity/Q7372011'), ('cname', "rowing at the 2000 Summer Olympics – lightweight women's double sculls")]
[('c', 'http://www.wikidata.org/entity/Q886703'), ('cname', "gymnastics at the 2000 Summer Olympics – men's vault")]
[('c', 'http://www.wikidata.org/entity/Q942211'), ('cname', "tennis at the 2000 Summer Olympics – women's doubles")]
[('c', 'http://www.wikidata.org/entity/Q942244'), ('cname', "tennis at the 2000 Summer Olympics – women's singles")]
[('c', 'http://www.wikidata.org/entity/Q7979999'), ('cname', "weightlifting at the 2000 Summer Olympics – men's 85 kg")]
[('c', 'http://www.wikidata.org/entity/Q1856784'), ('cname', "cycling at the 2000 Summer Olympics – men's cross-country")]
[('c', 'http://www.wikidata.org/entity/Q3879605'), ('cname', "swimming at the 2000 Summer Olympics – women's 4 × 100 metre medley relay")]


10

Try the reserve path: from Olympics to athletes.

In [9]:
#Q40
#start from Summer Olympics and check the connection with athletes
queryString = """
SELECT DISTINCT ?p ?pname
WHERE {
    
    wd:Q159821 wdt:P527* ?c.
    ?c ?p ?a.
    ?a wdt:P106/wdt:P279 wd:Q2066131. #athlete

    #labels
    ?p <http://schema.org/name> ?pname .
}
LIMIT 10
"""
print("Results")
run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1346'), ('pname', 'winner')]
[('p', 'http://www.wikidata.org/prop/direct/P545'), ('pname', 'torch lit by')]
[('p', 'http://www.wikidata.org/prop/direct/P710'), ('pname', 'participant')]
[('p', 'http://www.wikidata.org/prop/direct/P3279'), ('pname', 'statistical leader')]
[('p', 'http://www.wikidata.org/prop/direct/P543'), ('pname', 'oath made by')]
[('p', 'http://www.wikidata.org/prop/direct/P542'), ('pname', 'officially opened by')]
[('p', 'http://www.wikidata.org/prop/direct/P2321'), ('pname', 'general classification of race participants')]
[('p', 'http://www.wikidata.org/prop/direct/P3342'), ('pname', 'significant person')]


8

Retrieve some olympic competitions and their winners using P1346 found earlier.

In [14]:
#Q41
#start from Summer Olympics and check the connection with athletes
queryString = """
SELECT DISTINCT ?cname ?aname
WHERE {
    
    wd:Q159821 wdt:P527* ?c.
    ?c wdt:P1346 ?a.
    ?a wdt:P106/wdt:P279 wd:Q2066131. #athlete

    #labels
    ?c <http://schema.org/name> ?cname .
    ?a <http://schema.org/name> ?aname .
}
LIMIT 10
"""
print("Results")
run_query(queryString)

Results
[('cname', "wrestling at the 1952 Summer Olympics – men's Greco-Roman light heavyweight"), ('aname', 'Kelpo Gröndahl')]
[('cname', "wrestling at the 1948 Summer Olympics – men's Greco-Roman light heavyweight"), ('aname', 'Karl-Erik Nilsson')]
[('cname', "wrestling at the 1948 Summer Olympics – men's Greco-Roman featherweight"), ('aname', 'Mehmet Oktav')]
[('cname', "wrestling at the 1948 Summer Olympics – men's freestyle featherweight"), ('aname', 'Gazanfer Bilge')]
[('cname', "fencing at the 1948 Summer Olympics – men's sabre"), ('aname', 'Aladár Gerevich')]
[('cname', "shooting at the 1984 Summer Olympics – men's 50 metre rifle three positions"), ('aname', 'Malcolm Cooper')]
[('cname', "athletics at the 1984 Summer Olympics – women's long jump"), ('aname', 'Anișoara Cușmir-Stanciu')]
[('cname', "rowing at the 1984 Summer Olympics – men's coxless pair"), ('aname', 'Petru Iosub')]
[('cname', 'boxing at the 1952 Summer Olympics – light middleweight'), ('aname', 'László Papp')]
[

10

For each sport, retrieve how mnay french athletes won a gold medal .

In [15]:
#Q42
#count for each sport the number of french winners
queryString = """
SELECT (COUNT(?a) AS ?NumberOfWinners) (?s AS ?Sport)
WHERE {
    
    wd:Q159821 wdt:P527* ?c.
    ?c wdt:P1346 ?a; #winner
       wdt:P641  ?s. #sport
       
    ?a wdt:P106/wdt:P279 wd:Q2066131; #athlete
       wdt:P27 wd:Q142. 
}
GROUP BY (?s)
"""
print("Results")
run_query(queryString)

Results
[('NumberOfWinners', '9'), ('Sport', 'http://www.wikidata.org/entity/Q14085739')]
[('NumberOfWinners', '14'), ('Sport', 'http://www.wikidata.org/entity/Q206989')]
[('NumberOfWinners', '6'), ('Sport', 'http://www.wikidata.org/entity/Q108429')]
[('NumberOfWinners', '7'), ('Sport', 'http://www.wikidata.org/entity/Q159354')]
[('NumberOfWinners', '6'), ('Sport', 'http://www.wikidata.org/entity/Q213934')]
[('NumberOfWinners', '2'), ('Sport', 'http://www.wikidata.org/entity/Q42486')]
[('NumberOfWinners', '2'), ('Sport', 'http://www.wikidata.org/entity/Q215184')]
[('NumberOfWinners', '82'), ('Sport', 'http://www.wikidata.org/entity/Q221635')]
[('NumberOfWinners', '8'), ('Sport', 'http://www.wikidata.org/entity/Q847')]
[('NumberOfWinners', '4'), ('Sport', 'http://www.wikidata.org/entity/Q32112')]
[('NumberOfWinners', '1'), ('Sport', 'http://www.wikidata.org/entity/Q31874')]
[('NumberOfWinners', '34'), ('Sport', 'http://www.wikidata.org/entity/Q3609')]
[('NumberOfWinners', '1'), ('Sport'

21

Retrieve the sports where France has more winners than Germany.

In [6]:
#TASK 4.1: In which specific sport France has more winners than Germany
#compare french winners with german ones
queryString = """
SELECT ?Sport ?Sportname{
{
  SELECT (COUNT(DISTINCT ?f) AS ?NFrench) (?s AS ?Sport)
  WHERE {
    
      wd:Q159821 wdt:P527* ?c.
      ?c wdt:P1346 ?f; #winner
         wdt:P641  ?s. #sport
       
      ?f wdt:P106/wdt:P279 wd:Q2066131; #french
         wdt:P27 wd:Q142. 
  }
  GROUP BY (?s)
  }

  {
  SELECT (COUNT(DISTINCT ?g) AS ?NGerman) (?s AS ?Sport)
  WHERE {
    
      wd:Q159821 wdt:P527* ?c.
      ?c wdt:P1346 ?g; #winner
         wdt:P641  ?s. #sport
       
      ?g wdt:P106/wdt:P279 wd:Q2066131; #german
         wdt:P27 wd:Q183. 
  }
  GROUP BY (?s)
  }
  FILTER(?NFrench > ?NGerman)
  
   #labels
   ?Sport <http://schema.org/name> ?Sportname .
}
"""
print("Results")
run_query(queryString)

Results
[('Sport', 'http://www.wikidata.org/entity/Q847'), ('Sportname', 'tennis')]
[('Sport', 'http://www.wikidata.org/entity/Q11420'), ('Sportname', 'judo')]
[('Sport', 'http://www.wikidata.org/entity/Q12100'), ('Sportname', 'fencing')]
[('Sport', 'http://www.wikidata.org/entity/Q14085739'), ('Sportname', 'sailing')]
[('Sport', 'http://www.wikidata.org/entity/Q520611'), ('Sportname', 'mountain biking')]


5

Looking for female winners.

In [2]:
#Q43
#looking for female winners
queryString = """
SELECT DISTINCT ?cname ?aname ?a
WHERE {
    
    wd:Q159821 wdt:P527* ?c.
    ?c wdt:P1346 ?a.
    ?a wdt:P106/wdt:P279 wd:Q2066131. #athlete

    #labels
    ?c <http://schema.org/name> ?cname .
    ?a <http://schema.org/name> ?aname .
}
LIMIT 20
"""
print("Results")
run_query(queryString)

Results
[('cname', "wrestling at the 1952 Summer Olympics – men's Greco-Roman light heavyweight"), ('aname', 'Kelpo Gröndahl'), ('a', 'http://www.wikidata.org/entity/Q715090')]
[('cname', "wrestling at the 1948 Summer Olympics – men's Greco-Roman light heavyweight"), ('aname', 'Karl-Erik Nilsson'), ('a', 'http://www.wikidata.org/entity/Q645996')]
[('cname', "wrestling at the 1948 Summer Olympics – men's Greco-Roman featherweight"), ('aname', 'Mehmet Oktav'), ('a', 'http://www.wikidata.org/entity/Q710110')]
[('cname', "wrestling at the 1948 Summer Olympics – men's freestyle featherweight"), ('aname', 'Gazanfer Bilge'), ('a', 'http://www.wikidata.org/entity/Q1372262')]
[('cname', "fencing at the 1948 Summer Olympics – men's sabre"), ('aname', 'Aladár Gerevich'), ('a', 'http://www.wikidata.org/entity/Q312971')]
[('cname', "shooting at the 1984 Summer Olympics – men's 50 metre rifle three positions"), ('aname', 'Malcolm Cooper'), ('a', 'http://www.wikidata.org/entity/Q1360937')]
[('cname',

20

Looking for gender/sex property on Q448129 found above.

In [3]:
#Q44
#looking for gender/sex property on Q448129 found above
queryString = """
SELECT DISTINCT ?p ?pname
WHERE {
    
    wd:Q448129 ?p ?o.

    #labels
    ?p <http://schema.org/name> ?pname .
}
"""
print("Results")
run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P3408'), ('pname', 'FINA athlete ID')]
[('p', 'http://www.wikidata.org/prop/direct/P106'), ('pname', 'occupation')]
[('p', 'http://www.wikidata.org/prop/direct/P1344'), ('pname', 'participant in')]
[('p', 'http://www.wikidata.org/prop/direct/P1447'), ('pname', 'Sports-Reference.com Olympic athlete ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1741'), ('pname', 'GTAA ID')]
[('p', 'http://www.wikidata.org/prop/direct/P18'), ('pname', 'image')]
[('p', 'http://www.wikidata.org/prop/direct/P19'), ('pname', 'place of birth')]
[('p', 'http://www.wikidata.org/prop/direct/P2048'), ('pname', 'height')]
[('p', 'http://www.wikidata.org/prop/direct/P21'), ('pname', 'sex or gender')]
[('p', 'http://www.wikidata.org/prop/direct/P27'), ('pname', 'country of citizenship')]
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('pname', 'instance of')]
[('p', 'http://www.wikidata.org/prop/direct/P373'), ('pname', 'Commons category')]
[('p', 'http://w

18

The BGP to identify a female.

In [2]:
#Q45
#gender/sex property (P21)
queryString = """
SELECT DISTINCT ?o ?oname
WHERE {
    
    wd:Q448129 wdt:P21 ?o.

    #labels
    ?o <http://schema.org/name> ?oname .
}
LIMIT 10
"""
print("Results")
run_query(queryString)

Results
[('o', 'http://www.wikidata.org/entity/Q6581072'), ('oname', 'female')]


1

Retrieve some french female winners.

In [4]:
#Q46
#add female property (wdt:P21 wd:Q6581072) found above on Q42
queryString = """
SELECT DISTINCT ?a ?aname
WHERE {
    
    wd:Q159821 wdt:P527* ?c.
    ?c wdt:P1346 ?a; #winner
       wdt:P641  ?s. #sport
       
    ?a wdt:P106/wdt:P279 wd:Q2066131; #athlete
       wdt:P27 wd:Q142;               #french
       wdt:P21 wd:Q6581072.           #female
       
        #labels
        ?a <http://schema.org/name> ?aname .
}
LIMIT 10
"""
print("Results")
run_query(queryString)

Results
[('a', 'http://www.wikidata.org/entity/Q1528051'), ('aname', 'Marie Décugis')]
[('a', 'http://www.wikidata.org/entity/Q241577'), ('aname', 'Marguerite Broquedis')]
[('a', 'http://www.wikidata.org/entity/Q432388'), ('aname', 'Pascale Trinquet')]
[('a', 'http://www.wikidata.org/entity/Q201365'), ('aname', 'Micheline Ostermeyer')]
[('a', 'http://www.wikidata.org/entity/Q201365'), ('aname', 'Micheline Ostermeyer')]
[('a', 'http://www.wikidata.org/entity/Q201365'), ('aname', 'Micheline Ostermeyer')]
[('a', 'http://www.wikidata.org/entity/Q201365'), ('aname', 'Micheline Ostermeyer')]
[('a', 'http://www.wikidata.org/entity/Q432388'), ('aname', 'Pascale Trinquet')]
[('a', 'http://www.wikidata.org/entity/Q269722'), ('aname', 'Julie Bresset')]
[('a', 'http://www.wikidata.org/entity/Q45659'), ('aname', 'Laure Manaudou')]


10

In [5]:
#Q47
#add female property (wdt:P21 wd:Q6581072) found above on Q42
queryString = """
SELECT DISTINCT ?a ?aname
WHERE {
    
    wd:Q159821 wdt:P527* ?c.
    ?c wdt:P1346 ?a; #winner
       wdt:P641  ?s. #sport
       
    ?a wdt:P106/wdt:P279 wd:Q2066131; #athlete
       wdt:P27 wd:Q142;               #french
       wdt:P21 wd:Q6581072.           #female
       
        #labels
        ?a <http://schema.org/name> ?aname .
}
LIMIT 10
"""
print("Results")
run_query(queryString)

Results
[('a', 'http://www.wikidata.org/entity/Q1528051'), ('aname', 'Marie Décugis')]
[('a', 'http://www.wikidata.org/entity/Q241577'), ('aname', 'Marguerite Broquedis')]
[('a', 'http://www.wikidata.org/entity/Q432388'), ('aname', 'Pascale Trinquet')]
[('a', 'http://www.wikidata.org/entity/Q201365'), ('aname', 'Micheline Ostermeyer')]
[('a', 'http://www.wikidata.org/entity/Q269722'), ('aname', 'Julie Bresset')]
[('a', 'http://www.wikidata.org/entity/Q45659'), ('aname', 'Laure Manaudou')]
[('a', 'http://www.wikidata.org/entity/Q233488'), ('aname', 'Colette Besson')]
[('a', 'http://www.wikidata.org/entity/Q229085'), ('aname', 'Françoise Mbango Etone')]
[('a', 'http://www.wikidata.org/entity/Q240002'), ('aname', 'Lucie Décosse')]
[('a', 'http://www.wikidata.org/entity/Q232831'), ('aname', 'Suzanne Lenglen')]


10

Compare the number of female winners between France and Germany.

In [7]:
#TASK 4.2 Who has the highest number of female winners
#compare french winners with german ones
queryString = """
SELECT ?NFrench ?NGerman{
{
  SELECT (COUNT(DISTINCT ?f) AS ?NFrench)
  WHERE {
    
      wd:Q159821 wdt:P527* ?c.
      ?c wdt:P1346 ?f. #winner
       
      ?f wdt:P106/wdt:P279 wd:Q2066131; #french
         wdt:P27 wd:Q142;
         wdt:P21 wd:Q6581072.           #female
  }

  }

  {
  SELECT (COUNT( DISTINCT ?g) AS ?NGerman)
  WHERE {
    
      wd:Q159821 wdt:P527* ?c.
      ?c wdt:P1346 ?g. #winner
       
      ?g wdt:P106/wdt:P279 wd:Q2066131; #german
         wdt:P27 wd:Q183;
         wdt:P21 wd:Q6581072.           #female
  }

  }  
  
}
"""
print("Results")
run_query(queryString)

Results
[('NFrench', '30'), ('NGerman', '98')]


1

Look if France has more female winners than Germany in any sport.

In [10]:
#ADDITIONAL ON TASK 4.2: looking if french has more female winners in any sport
queryString = """
SELECT ?Sport ?Sportname{
{
  SELECT (COUNT(DISTINCT ?f) AS ?NFrench) (?s AS ?Sport)
  WHERE {
    
      wd:Q159821 wdt:P527* ?c.
      ?c wdt:P1346 ?f; #winner
         wdt:P641  ?s. #sport
       
      ?f wdt:P106/wdt:P279 wd:Q2066131; #french
         wdt:P27 wd:Q142;
         wdt:P21 wd:Q6581072.           #female
  }
  GROUP BY (?s)
  }

  {
  SELECT (COUNT(DISTINCT ?g) AS ?NGerman) (?s AS ?Sport)
  WHERE {
    
      wd:Q159821 wdt:P527* ?c.
      ?c wdt:P1346 ?g; #winner
         wdt:P641  ?s. #sport
       
      ?g wdt:P106/wdt:P279 wd:Q2066131; #german
         wdt:P27 wd:Q183;
         wdt:P21 wd:Q6581072.           #female
  }
  GROUP BY (?s)
  }
  FILTER(?NFrench > ?NGerman)
  
   #labels
   ?Sport <http://schema.org/name> ?Sportname .
}
"""
print("Results")
run_query(queryString)

Results
[('Sport', 'http://www.wikidata.org/entity/Q11420'), ('Sportname', 'judo')]


1

In [11]:
#Q48
#in previous task we found the BGP for participating in competition

queryString = """
SELECT DISTINCT ?a ?aname ?oname
WHERE {
       
    ?a wdt:P106/wdt:P279 wd:Q2066131; #athlete
       wdt:P27 wd:Q142;               #french
       wdt:P710|wdt:P1344 ?o.
       
       ?o wdt:P31 wd:Q159821.
       
        #labels
        ?a <http://schema.org/name> ?aname .
        ?o <http://schema.org/name> ?oname .
}
LIMIT 10
"""
print("Results")
run_query(queryString)

Results
[('a', 'http://www.wikidata.org/entity/Q13639919'), ('aname', 'Joseph Goubert'), ('oname', '1936 Summer Olympics')]
[('a', 'http://www.wikidata.org/entity/Q30133063'), ('aname', 'Paul Bronner'), ('oname', '1936 Summer Olympics')]
[('a', 'http://www.wikidata.org/entity/Q5079280'), ('aname', 'Charles Imbault'), ('oname', '1936 Summer Olympics')]
[('a', 'http://www.wikidata.org/entity/Q5373240'), ('aname', 'Emmanuel Gonat'), ('oname', '1936 Summer Olympics')]
[('a', 'http://www.wikidata.org/entity/Q168971'), ('aname', 'Robert Dorgebray'), ('oname', '1936 Summer Olympics')]
[('a', 'http://www.wikidata.org/entity/Q2356350'), ('aname', 'René Bougnol'), ('oname', '1936 Summer Olympics')]
[('a', 'http://www.wikidata.org/entity/Q2357548'), ('aname', 'René Bondoux'), ('oname', '1936 Summer Olympics')]
[('a', 'http://www.wikidata.org/entity/Q3092463'), ('aname', 'Félix Grimonprez'), ('oname', '1936 Summer Olympics')]
[('a', 'http://www.wikidata.org/entity/Q2960736'), ('aname', 'Charles de

10

In [12]:
#Q49
#in previous task we found the BGP for participating in competition
queryString = """
SELECT ?a ?aname ?oname
WHERE {
       
    ?a wdt:P106/wdt:P279 wd:Q2066131; #athlete
       wdt:P27 wd:Q142;               #french
       wdt:P710|wdt:P1344 ?o.
       
       ?o wdt:P31 wd:Q159821.
       
        #labels
        ?a <http://schema.org/name> ?aname .
        ?o <http://schema.org/name> ?oname .
}
LIMIT 10
"""
print("Results")
run_query(queryString)

Results
[('a', 'http://www.wikidata.org/entity/Q13639919'), ('aname', 'Joseph Goubert'), ('oname', '1936 Summer Olympics')]
[('a', 'http://www.wikidata.org/entity/Q30133063'), ('aname', 'Paul Bronner'), ('oname', '1936 Summer Olympics')]
[('a', 'http://www.wikidata.org/entity/Q5079280'), ('aname', 'Charles Imbault'), ('oname', '1936 Summer Olympics')]
[('a', 'http://www.wikidata.org/entity/Q5373240'), ('aname', 'Emmanuel Gonat'), ('oname', '1936 Summer Olympics')]
[('a', 'http://www.wikidata.org/entity/Q168971'), ('aname', 'Robert Dorgebray'), ('oname', '1936 Summer Olympics')]
[('a', 'http://www.wikidata.org/entity/Q2356350'), ('aname', 'René Bougnol'), ('oname', '1936 Summer Olympics')]
[('a', 'http://www.wikidata.org/entity/Q2357548'), ('aname', 'René Bondoux'), ('oname', '1936 Summer Olympics')]
[('a', 'http://www.wikidata.org/entity/Q3092463'), ('aname', 'Félix Grimonprez'), ('oname', '1936 Summer Olympics')]
[('a', 'http://www.wikidata.org/entity/Q2960736'), ('aname', 'Charles de

10

In [13]:
#Q50
#check a specific entity found above
queryString = """
SELECT ?o ?oname
WHERE {
       
    wd:Q13639919 wdt:P710|wdt:P1344 ?o.     
    ?o wdt:P31 wd:Q159821.
       
        #labels
        ?o <http://schema.org/name> ?oname .
}
LIMIT 10
"""
print("Results")
run_query(queryString)

Results
[('o', 'http://www.wikidata.org/entity/Q8150'), ('oname', '1936 Summer Olympics')]


1

In [14]:
#Q51
#check a specific entity found above
queryString = """
SELECT ?p ?pname
WHERE {
       
    wd:Q13639919 wdt:P710|wdt:P1344 ?o.     
    ?o ?p wd:Q159821.
       
        #labels
        ?p <http://schema.org/name> ?pname .
}
LIMIT 10
"""
print("Results")
run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('pname', 'instance of')]


1

Retrieve the french athletes in order of participations in Olympics.  

In [16]:
#Q52
#count the number of participation in olympic games for french athletes
#retrieve top 10
queryString = """
SELECT (COUNT(?o) AS ?NumberOfParticipation) ?a
WHERE {
       
    ?a wdt:P106/wdt:P279 wd:Q2066131; #athlete
       wdt:P27 wd:Q142;               #french
       wdt:P710|wdt:P1344 ?o.
       
    ?o wdt:P31 wd:Q159821.
}
GROUP BY (?a)
ORDER BY DESC(?NumberOfParticipation)
LIMIT 10
"""
print("Results")
run_query(queryString)

Results
[('NumberOfParticipation', '12'), ('a', 'http://www.wikidata.org/entity/Q3379370')]
[('NumberOfParticipation', '8'), ('a', 'http://www.wikidata.org/entity/Q316224')]
[('NumberOfParticipation', '8'), ('a', 'http://www.wikidata.org/entity/Q1605975')]
[('NumberOfParticipation', '7'), ('a', 'http://www.wikidata.org/entity/Q233834')]
[('NumberOfParticipation', '6'), ('a', 'http://www.wikidata.org/entity/Q3367302')]
[('NumberOfParticipation', '6'), ('a', 'http://www.wikidata.org/entity/Q3052611')]
[('NumberOfParticipation', '6'), ('a', 'http://www.wikidata.org/entity/Q3082493')]
[('NumberOfParticipation', '6'), ('a', 'http://www.wikidata.org/entity/Q3175379')]
[('NumberOfParticipation', '6'), ('a', 'http://www.wikidata.org/entity/Q2863132')]
[('NumberOfParticipation', '6'), ('a', 'http://www.wikidata.org/entity/Q2609559')]


10

In [17]:
#Q53
#check a specific entity found above
queryString = """
SELECT ?o ?oname
WHERE {
       
    wd:Q3379370 wdt:P710|wdt:P1344 ?o.     
    ?o wdt:P31 wd:Q159821.
       
    #labels
    ?o <http://schema.org/name> ?oname .
}
LIMIT 25
"""
print("Results")
run_query(queryString)

Results
[('o', 'http://www.wikidata.org/entity/Q8470'), ('oname', '1988 Summer Olympics')]
[('o', 'http://www.wikidata.org/entity/Q8488'), ('oname', '1992 Summer Olympics')]
[('o', 'http://www.wikidata.org/entity/Q8531'), ('oname', '1996 Summer Olympics')]
[('o', 'http://www.wikidata.org/entity/Q8544'), ('oname', '2000 Summer Olympics')]
[('o', 'http://www.wikidata.org/entity/Q8450'), ('oname', '1980 Summer Olympics')]
[('o', 'http://www.wikidata.org/entity/Q8456'), ('oname', '1984 Summer Olympics')]


6

Add distinct in the previous query.

In [18]:
#Q54
#add distinct
queryString = """
SELECT (COUNT(DISTINCT ?o) AS ?NumberOfParticipation) ?a
WHERE {
       
    ?a wdt:P106/wdt:P279 wd:Q2066131; #athlete
       wdt:P27 wd:Q142;               #french
       wdt:P710|wdt:P1344 ?o.
       
    ?o wdt:P31 wd:Q159821.
}
GROUP BY (?a)
ORDER BY DESC(?NumberOfParticipation)
LIMIT 10
"""
print("Results")
run_query(queryString)

Results
[('NumberOfParticipation', '7'), ('a', 'http://www.wikidata.org/entity/Q233834')]
[('NumberOfParticipation', '6'), ('a', 'http://www.wikidata.org/entity/Q3379370')]
[('NumberOfParticipation', '5'), ('a', 'http://www.wikidata.org/entity/Q2493829')]
[('NumberOfParticipation', '5'), ('a', 'http://www.wikidata.org/entity/Q2359387')]
[('NumberOfParticipation', '5'), ('a', 'http://www.wikidata.org/entity/Q239263')]
[('NumberOfParticipation', '5'), ('a', 'http://www.wikidata.org/entity/Q3159341')]
[('NumberOfParticipation', '5'), ('a', 'http://www.wikidata.org/entity/Q934015')]
[('NumberOfParticipation', '5'), ('a', 'http://www.wikidata.org/entity/Q144864')]
[('NumberOfParticipation', '4'), ('a', 'http://www.wikidata.org/entity/Q949468')]
[('NumberOfParticipation', '4'), ('a', 'http://www.wikidata.org/entity/Q1805125')]


10

Retrieve the german athletes with the highest number of olympic participations.

In [19]:
#TASK 4.3 For each country , which are the atheletes with the highest number of participations 
#same as above but for german ones
queryString = """
SELECT (COUNT(DISTINCT ?o) AS ?NumberOfParticipation) ?a
WHERE {
       
    ?a wdt:P106/wdt:P279 wd:Q2066131; #athlete
       wdt:P27 wd:Q183;               #german
       wdt:P710|wdt:P1344 ?o.
       
    ?o wdt:P31 wd:Q159821.
}
GROUP BY (?a)
ORDER BY DESC(?NumberOfParticipation)
LIMIT 10
"""
print("Results")
run_query(queryString)

Results
[('NumberOfParticipation', '7'), ('a', 'http://www.wikidata.org/entity/Q248926')]
[('NumberOfParticipation', '7'), ('a', 'http://www.wikidata.org/entity/Q553279')]
[('NumberOfParticipation', '6'), ('a', 'http://www.wikidata.org/entity/Q70562')]
[('NumberOfParticipation', '5'), ('a', 'http://www.wikidata.org/entity/Q73336')]
[('NumberOfParticipation', '5'), ('a', 'http://www.wikidata.org/entity/Q506576')]
[('NumberOfParticipation', '5'), ('a', 'http://www.wikidata.org/entity/Q1885348')]
[('NumberOfParticipation', '5'), ('a', 'http://www.wikidata.org/entity/Q544298')]
[('NumberOfParticipation', '5'), ('a', 'http://www.wikidata.org/entity/Q453208')]
[('NumberOfParticipation', '5'), ('a', 'http://www.wikidata.org/entity/Q384411')]
[('NumberOfParticipation', '5'), ('a', 'http://www.wikidata.org/entity/Q61161')]


10

In [20]:
#Q55
#in previous task we found the BGP for participating in generic competition instead of olympic ones
queryString = """
SELECT ?a ?aname ?o ?oname
WHERE {
       
    ?a wdt:P106/wdt:P279 wd:Q2066131; #athlete
       wdt:P27 wd:Q142;               #french
       wdt:P710|wdt:P1344 ?o.
       
       ?o wdt:P641 ?sport.
       
        #labels
        ?a <http://schema.org/name> ?aname .
        ?o <http://schema.org/name> ?oname .
}
LIMIT 10
"""
print("Results")
run_query(queryString)

Results
[('a', 'http://www.wikidata.org/entity/Q3568815'), ('aname', 'Charles William Martin'), ('o', 'http://www.wikidata.org/entity/Q859344'), ('oname', 'sailing at the 1900 Summer Olympics – .5 to 1 ton')]
[('a', 'http://www.wikidata.org/entity/Q15973499'), ('aname', 'Rudy Barbier'), ('o', 'http://www.wikidata.org/entity/Q1280387'), ('oname', '2013 UCI Europe Tour')]
[('a', 'http://www.wikidata.org/entity/Q15973499'), ('aname', 'Rudy Barbier'), ('o', 'http://www.wikidata.org/entity/Q15041668'), ('oname', '2014 UCI Europe Tour')]
[('a', 'http://www.wikidata.org/entity/Q15973499'), ('aname', 'Rudy Barbier'), ('o', 'http://www.wikidata.org/entity/Q18342122'), ('oname', '2015 UCI Europe Tour')]
[('a', 'http://www.wikidata.org/entity/Q15973499'), ('aname', 'Rudy Barbier'), ('o', 'http://www.wikidata.org/entity/Q21029681'), ('oname', '2016 UCI Europe Tour')]
[('a', 'http://www.wikidata.org/entity/Q15973499'), ('aname', 'Rudy Barbier'), ('o', 'http://www.wikidata.org/entity/Q30750603'), ('

10

Try to obtain the participation in a single olympic event instead of a olympic games edition.

In [21]:
#Q56
#try to obtain the participation in a single olympic event instead of a olympic games edition
queryString = """
SELECT ?p ?pname ?o ?oname
WHERE {

    wd:Q859344 ?p ?o.
          
    #labels
    ?p <http://schema.org/name> ?pname .
    ?o <http://schema.org/name> ?oname .
}
LIMIT 30
"""
print("Results")
run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P276'), ('pname', 'location'), ('o', 'http://www.wikidata.org/entity/Q275371'), ('oname', 'Meulan-en-Yvelines')]
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('pname', 'instance of'), ('o', 'http://www.wikidata.org/entity/Q18536594'), ('oname', 'Olympic sporting event')]
[('p', 'http://www.wikidata.org/prop/direct/P641'), ('pname', 'sport'), ('o', 'http://www.wikidata.org/entity/Q14085739'), ('oname', 'sailing')]
[('p', 'http://www.wikidata.org/prop/direct/P1346'), ('pname', 'winner'), ('o', 'http://www.wikidata.org/entity/Q1802560'), ('oname', 'Lorne Currie')]
[('p', 'http://www.wikidata.org/prop/direct/P361'), ('pname', 'part of'), ('o', 'http://www.wikidata.org/entity/Q900212'), ('oname', 'sailing at the 1900 Summer Olympics')]


5

In [22]:
#Q57
#use Q18536594
queryString = """
SELECT ?a ?aname ?o ?oname
WHERE {
       
    ?a wdt:P106/wdt:P279 wd:Q2066131; #athlete
       wdt:P27 wd:Q142;               #french
       wdt:P710|wdt:P1344 ?o.
       
       ?o wdt:P31 wd:Q18536594.
       
        #labels
        ?a <http://schema.org/name> ?aname .
        ?o <http://schema.org/name> ?oname .
}
LIMIT 10
"""
print("Results")
run_query(queryString)

Results
[('a', 'http://www.wikidata.org/entity/Q26252931'), ('aname', 'Jean-Pierre Bourhis'), ('o', 'http://www.wikidata.org/entity/Q26212275'), ('oname', "canoeing at the 2016 Summer Olympics – men's slalom C-1")]
[('a', 'http://www.wikidata.org/entity/Q26252931'), ('aname', 'Jean-Pierre Bourhis'), ('o', 'http://www.wikidata.org/entity/Q76711065'), ('oname', "canoeing at the 2020 Summer Olympics – men's slalom C-1")]
[('a', 'http://www.wikidata.org/entity/Q41111427'), ('aname', 'Martin Thomas'), ('o', 'http://www.wikidata.org/entity/Q76711065'), ('oname', "canoeing at the 2020 Summer Olympics – men's slalom C-1")]
[('a', 'http://www.wikidata.org/entity/Q316224'), ('aname', 'Tony Estanguet'), ('o', 'http://www.wikidata.org/entity/Q3491894'), ('oname', "canoeing at the 2004 Summer Olympics – men's slalom C-1")]
[('a', 'http://www.wikidata.org/entity/Q8078410'), ('aname', 'Éric le Leuch'), ('o', 'http://www.wikidata.org/entity/Q3655475'), ('oname', "canoeing at the 2000 Summer Olympics –

10

Retrieve the athletes with the highest number of participations in olympic competitions instead of Olympic Games (that i have done some cells above).

In [23]:
#TASK 4.3 For each country , which are the atheletes with the highest number of participations
#consider participation in single olympic events instead of editions
#same as above but for german ones
queryString = """
SELECT (COUNT(DISTINCT ?o) AS ?NumberOfParticipation) ?a
WHERE {
       
    ?a wdt:P106/wdt:P279 wd:Q2066131; #athlete
       wdt:P27 wd:Q183;               #german
       wdt:P710|wdt:P1344 ?o.
       
    ?o wdt:P31 wd:Q18536594.
}
GROUP BY (?a)
ORDER BY DESC(?NumberOfParticipation)
LIMIT 10
"""
print("Results")
run_query(queryString)

Results
[('NumberOfParticipation', '13'), ('a', 'http://www.wikidata.org/entity/Q2151006')]
[('NumberOfParticipation', '12'), ('a', 'http://www.wikidata.org/entity/Q57747')]
[('NumberOfParticipation', '10'), ('a', 'http://www.wikidata.org/entity/Q62968')]
[('NumberOfParticipation', '10'), ('a', 'http://www.wikidata.org/entity/Q473760')]
[('NumberOfParticipation', '10'), ('a', 'http://www.wikidata.org/entity/Q163055')]
[('NumberOfParticipation', '10'), ('a', 'http://www.wikidata.org/entity/Q62501')]
[('NumberOfParticipation', '10'), ('a', 'http://www.wikidata.org/entity/Q57582')]
[('NumberOfParticipation', '9'), ('a', 'http://www.wikidata.org/entity/Q567102')]
[('NumberOfParticipation', '8'), ('a', 'http://www.wikidata.org/entity/Q882906')]
[('NumberOfParticipation', '8'), ('a', 'http://www.wikidata.org/entity/Q64106')]


10

In [24]:
#Q58
#check a specific entity found above
queryString = """
SELECT ?o ?oname
WHERE {
       
    wd:Q2151006 wdt:P710|wdt:P1344 ?o.     
    ?o wdt:P31 wd:Q18536594.
       
    #labels
    ?o <http://schema.org/name> ?oname .
}
LIMIT 25
"""
print("Results")
run_query(queryString)

Results
[('o', 'http://www.wikidata.org/entity/Q5033039'), ('oname', "canoeing at the 1988 Summer Olympics – women's K-4 500 metres")]
[('o', 'http://www.wikidata.org/entity/Q5033052'), ('oname', "canoeing at the 1992 Summer Olympics – women's K-1 500 metres")]
[('o', 'http://www.wikidata.org/entity/Q5033056'), ('oname', "canoeing at the 1992 Summer Olympics – women's K-4 500 metres")]
[('o', 'http://www.wikidata.org/entity/Q5033069'), ('oname', "canoeing at the 1996 Summer Olympics – women's K-1 500 metres")]
[('o', 'http://www.wikidata.org/entity/Q5033087'), ('oname', "canoeing at the 2000 Summer Olympics – women's K-4 500 metres")]
[('o', 'http://www.wikidata.org/entity/Q1222361'), ('oname', "canoeing at the 2004 Summer Olympics – women's K-2 500 metres")]
[('o', 'http://www.wikidata.org/entity/Q1222367'), ('oname', "canoeing at the 2004 Summer Olympics – women's K-4 500 metres")]
[('o', 'http://www.wikidata.org/entity/Q5033013'), ('oname', "canoeing at the 1980 Summer Olympics – wo

13

In [25]:
#TASK 4.3 For each country , which are the atheletes with the highest number of participations
#same as above but for french ones
queryString = """
SELECT (COUNT(DISTINCT ?o) AS ?NumberOfParticipation) ?a
WHERE {
       
    ?a wdt:P106/wdt:P279 wd:Q2066131; #athlete
       wdt:P27 wd:Q142;               #german
       wdt:P710|wdt:P1344 ?o.
       
    ?o wdt:P31 wd:Q18536594.
}
GROUP BY (?a)
ORDER BY DESC(?NumberOfParticipation)
LIMIT 10
"""
print("Results")
run_query(queryString)

Results
[('NumberOfParticipation', '11'), ('a', 'http://www.wikidata.org/entity/Q129192')]
[('NumberOfParticipation', '8'), ('a', 'http://www.wikidata.org/entity/Q1392951')]
[('NumberOfParticipation', '7'), ('a', 'http://www.wikidata.org/entity/Q44413')]
[('NumberOfParticipation', '7'), ('a', 'http://www.wikidata.org/entity/Q1334464')]
[('NumberOfParticipation', '7'), ('a', 'http://www.wikidata.org/entity/Q229002')]
[('NumberOfParticipation', '7'), ('a', 'http://www.wikidata.org/entity/Q242959')]
[('NumberOfParticipation', '6'), ('a', 'http://www.wikidata.org/entity/Q1392625')]
[('NumberOfParticipation', '6'), ('a', 'http://www.wikidata.org/entity/Q3164706')]
[('NumberOfParticipation', '6'), ('a', 'http://www.wikidata.org/entity/Q27820')]
[('NumberOfParticipation', '6'), ('a', 'http://www.wikidata.org/entity/Q3098966')]


10

<h3> TASK 4 ENDS HERE </h3>
In this task I found the sports where french athlets are better than german one, the sports where female french athletes are better than germans, the number of female winners for both countries. On top of that I found the athletes who took part most times in Olympic Games and also the ones who participated in more olympic events.